In [ ]:
!pip install pulp

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
import string
from datetime import datetime, timedelta
import datetime

In [ ]:

flight_schedule = pd.read_excel("flights_schedule.xlsx")  # Read Excel file
aircraft_type = pd.read_excel("Aircraft_type.xlsx")
airport_code = pd.read_excel("Airport_code.xlsx")
salary = pd.read_excel("salary.xlsx")

In [ ]:
flight_schedule

,ID,From,To,Start_time(SLST),Arrival_time(SLST),Aircraft_Type,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,UL 173,Colombo,Bengaluru,01:10:00,02:35:00,A320,0,1,1,1,0,1,0
1,UL 306,Colombo,Singapore,01:10:00,10:10:00,A320,0,1,0,1,1,0,0
2,UL 174,Bengaluru,Colombo,03:35:00,05:05:00,A320,0,1,1,1,0,1,0
3,UL 307,Singapore,Male,09:45:00,14:25:00,A330,0,1,0,0,1,0,0
4,UL 102,Male,Colombo,09:55:00,11:20:00,A330,1,1,1,1,1,1,1
5,UL 506,London,Colombo,18:05:00,04:25:00,A330,0,0,0,0,0,1,1


In [ ]:
aircraft_type

,aircraft_type,Min,Max
0,A320,3,50
1,A330,6,50
2,A300,6,50


In [ ]:
airport_code

,City,IATA airport code
0,Colombo,CMB
1,Bengaluru,BLR
2,Singapore,SIN
3,Male,MLE
4,London,LHR


In [ ]:
# Add IATA codes to the Flights DataFrame
flight_schedule = flight_schedule.merge(airport_code.rename(columns={'City': 'From', 'IATA airport code': 'From_IATA'}),
                        on='From', how='left')

flight_schedule = flight_schedule.merge(airport_code.rename(columns={'City': 'To', 'IATA airport code': 'To_IATA'}),
                        on='To', how='left')

In [ ]:
flight_schedule

,ID,From,To,Start_time(SLST),Arrival_time(SLST),Aircraft_Type,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,From_IATA,To_IATA
0,UL 173,Colombo,Bengaluru,01:10:00,02:35:00,A320,0,1,1,1,0,1,0,CMB,BLR
1,UL 306,Colombo,Singapore,01:10:00,10:10:00,A320,0,1,0,1,1,0,0,CMB,SIN
2,UL 174,Bengaluru,Colombo,03:35:00,05:05:00,A320,0,1,1,1,0,1,0,BLR,CMB
3,UL 307,Singapore,Male,09:45:00,14:25:00,A330,0,1,0,0,1,0,0,SIN,MLE
4,UL 102,Male,Colombo,09:55:00,11:20:00,A330,1,1,1,1,1,1,1,MLE,CMB
5,UL 506,London,Colombo,18:05:00,04:25:00,A330,0,0,0,0,0,1,1,LHR,CMB


In [ ]:
# Merge the dataframes on Aircraft Type
flight_schedule = flight_schedule.merge(aircraft_type, left_on="Aircraft_Type", right_on="aircraft_type", how="left")

# Drop the redundant aircraft_type column
flight_schedule.drop(columns=["aircraft_type"], inplace=True)

In [ ]:
flight_schedule.head()

,ID,From,To,Start_time(SLST),Arrival_time(SLST),Aircraft_Type,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,From_IATA,To_IATA,Min,Max
0,UL 173,Colombo,Bengaluru,01:10:00,02:35:00,A320,0,1,1,1,0,1,0,CMB,BLR,3,50
1,UL 306,Colombo,Singapore,01:10:00,10:10:00,A320,0,1,0,1,1,0,0,CMB,SIN,3,50
2,UL 174,Bengaluru,Colombo,03:35:00,05:05:00,A320,0,1,1,1,0,1,0,BLR,CMB,3,50
3,UL 307,Singapore,Male,09:45:00,14:25:00,A330,0,1,0,0,1,0,0,SIN,MLE,6,50
4,UL 102,Male,Colombo,09:55:00,11:20:00,A330,1,1,1,1,1,1,1,MLE,CMB,6,50


In [ ]:
# Convert times to datetime, but keep only time (remove the date part)
flight_schedule['Start_time(SLST)'] = pd.to_datetime(flight_schedule['Start_time(SLST)'], format='%H:%M:%S').dt.strftime('%H:%M:%S')
flight_schedule['Arrival_time(SLST)'] = pd.to_datetime(flight_schedule['Arrival_time(SLST)'], format='%H:%M:%S').dt.strftime('%H:%M:%S')

# Function to calculate duration in hours (as a decimal)
def calculate_duration(row):
    start_time = pd.to_datetime(row['Start_time(SLST)'], format='%H:%M:%S')
    arrival_time = pd.to_datetime(row['Arrival_time(SLST)'], format='%H:%M:%S')

    if arrival_time > start_time:
        duration = arrival_time - start_time
    else:
        # If arrival is the next day
        arrival_time += pd.Timedelta(hours=24)
        duration = arrival_time - start_time

    hours = duration.seconds // 3600
    minutes = (duration.seconds % 3600) // 60
    # Convert minutes to decimal and add to hours
    decimal_hours = hours + minutes / 60
    return round(decimal_hours, 2)

# Apply the function to calculate the duration for each flight
flight_schedule['Duration_hours'] = flight_schedule.apply(calculate_duration, axis=1)
flight_schedule.head()

,ID,From,To,Start_time(SLST),Arrival_time(SLST),Aircraft_Type,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,From_IATA,To_IATA,Min,Max,Duration_hours
0,UL 173,Colombo,Bengaluru,01:10:00,02:35:00,A320,0,1,1,1,0,1,0,CMB,BLR,3,50,1.42
1,UL 306,Colombo,Singapore,01:10:00,10:10:00,A320,0,1,0,1,1,0,0,CMB,SIN,3,50,9.00
2,UL 174,Bengaluru,Colombo,03:35:00,05:05:00,A320,0,1,1,1,0,1,0,BLR,CMB,3,50,1.50
3,UL 307,Singapore,Male,09:45:00,14:25:00,A330,0,1,0,0,1,0,0,SIN,MLE,6,50,4.67
4,UL 102,Male,Colombo,09:55:00,11:20:00,A330,1,1,1,1,1,1,1,MLE,CMB,6,50,1.42


In [ ]:
# Transforming data into a single table with a 'Day' column
all_flights = []
for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
    temp_df = flight_schedule[flight_schedule[day] == 1][["ID", "From","From_IATA", "To","To_IATA","Start_time(SLST)", "Arrival_time(SLST)","Min", "Max", "Duration_hours"]].copy()
    temp_df.insert(0, "Start_Day", day)  # Insert 'Day' column at the beginning
    all_flights.append(temp_df)

final_df = pd.concat(all_flights, ignore_index=True)
# final_df


# # Save the final schedule to a new Excel file
# output_file = "grouped_flights.xlsx"
# final_df.to_excel(output_file, index=False)

# Displaying the final schedule
# print(final_df.to_string(index=False))
final_df

,Start_Day,ID,From,From_IATA,To,To_IATA,Start_time(SLST),Arrival_time(SLST),Min,Max,Duration_hours
0,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42
1,Tuesday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42
2,Tuesday,UL 306,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,9.00
3,Tuesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50
4,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,4.67
5,Tuesday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42
6,Wednesday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42
7,Wednesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50
8,Wednesday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42
9,Thursday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42


Get the end day

In [ ]:
# Convert 'start time(SLST)' to a datetime object for proper sorting
final_df["Start_time(SLST)"] = pd.to_datetime(final_df["Start_time(SLST)"], format="%H:%M:%S").dt.time

# Define a custom sorting order for days of the week
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Sort by the custom day order first, then by 'start time(SLST)'
final_df["Start_Day"] = pd.Categorical(final_df["Start_Day"], categories=day_order, ordered=True)
final_df = final_df.sort_values(by=["Start_Day", "Start_time(SLST)"]).reset_index(drop=True)

# Display the sorted dataframe
# final_df

In [ ]:
from datetime import datetime, timedelta

# Assume all the flights end in 2 days
travel_duration = final_df['Duration_hours'].tolist()  # Convert to list if it's a pandas Series
# travel_start_time = [datetime.strptime(time, "%H:%M:%S") for time in final_df['Start_time(SLST)']]  # Create list of start times
# Create a reference date (you can use any date)
reference_date = datetime(2000, 1, 1)

# Create list of start times as datetime objects
travel_start_time = [datetime.combine(reference_date, time) for time in final_df['Start_time(SLST)']]

travel_endtime_duration_variable = []
time_to_add = []

for i in range(len(final_df)):
    time_to_add.append(timedelta(hours=travel_duration[i]))  # Append timedelta to the list
    new_time = travel_start_time[i] + time_to_add[i]

    # Check if the new time crosses into the next day (or days)
    if new_time.hour < travel_start_time[i].hour:  # This checks if the new time has rolled over past midnight
        travel_endtime_duration_variable.append(1)
    else:
        travel_endtime_duration_variable.append(0)

# Define a custom sorting order for days of the week
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

In [ ]:
index = []
for i in range(len(final_df)):
  travel_start_day=final_df['Start_Day'][i];
  if travel_start_day in day_order:
     index.append(day_order.index(travel_start_day))
  else:
      index.append(-1)  # or handle the case where travel_start_day is not in day_order

In [ ]:
travel_end_day_index=[]
for i in range(len(final_df)):
  travel_end_day_index.append(travel_endtime_duration_variable[i] + index[i])
  if  travel_end_day_index[i] > 6:
    travel_end_day_index[i] = travel_end_day_index[i] - 7

In [ ]:
End_days = [day_order[index] for index in travel_end_day_index]
final_df['End_Day'] = End_days

In [ ]:
final_2 = final_df.copy(deep=True)
final_3 = final_df.copy(deep=True)

In [ ]:
final_df

,Start_Day,ID,From,From_IATA,To,To_IATA,Start_time(SLST),Arrival_time(SLST),Min,Max,Duration_hours,End_Day
0,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42,Monday
1,Tuesday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Tuesday
2,Tuesday,UL 306,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,9.00,Tuesday
3,Tuesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Tuesday
4,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,4.67,Tuesday
5,Tuesday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42,Tuesday
6,Wednesday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Wednesday
7,Wednesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Wednesday
8,Wednesday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42,Wednesday
9,Thursday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Thursday


In [ ]:
# Assign Allocated_Members dynamically without storing Day_Frequency - Flight 1
final_df["Allocated_Members_Flight_1"] = final_df["Start_Day"].str[0:2] + "/" + final_df["From_IATA"].str[0:3] + "/"+ final_df["ID"].str[-3:] + "/"+ final_df["To_IATA"].str[0:3]
final_df

,Start_Day,ID,From,From_IATA,To,To_IATA,Start_time(SLST),Arrival_time(SLST),Min,Max,Duration_hours,End_Day,Allocated_Members_Flight_1
0,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42,Monday,Mo/MLE/102/CMB
1,Tuesday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Tuesday,Tu/CMB/173/BLR
2,Tuesday,UL 306,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,9.00,Tuesday,Tu/CMB/306/SIN
3,Tuesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Tuesday,Tu/BLR/174/CMB
4,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,4.67,Tuesday,Tu/SIN/307/MLE
5,Tuesday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42,Tuesday,Tu/MLE/102/CMB
6,Wednesday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Wednesday,We/CMB/173/BLR
7,Wednesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Wednesday,We/BLR/174/CMB
8,Wednesday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42,Wednesday,We/MLE/102/CMB
9,Thursday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Thursday,Th/CMB/173/BLR


In [ ]:
unique_count =final_df["Allocated_Members_Flight_1"].nunique()

In [ ]:
unique_count

22

In [ ]:
# Assign Allocated_Members dynamically without storing Day_Frequency - Flight 1
# final_df["Allocated_Members_Flight_1"] =final_df["From_IATA"].str[0:3] + "_" + final_df["To_IATA"].str[0:3] +  "_" + final_df["Start_Day"].str[0:2] + (final_df.groupby("Start_Day").cumcount() + 1).astype(str)
# final_df

In [ ]:
Flight_1_df = final_df

In [ ]:
Flight_1_df.columns = ["Flight_1_Start_Day", "Flight_1_ID", "Flight_1_From", "Flight_1_From_IATA", "Flight_1_To","Flight_1_To_IATA", "Flight_1_start_time(SLST)", "Flight_1_Arrival_time(SLST)", "Flight_1_Min",	"Flight_1_Max", "Flight_1_duration_hours", "Flight_1_End_Day", "Flight_1_Allocated_Members"]

In [ ]:
Flight_1_df.head()

,Flight_1_Start_Day,Flight_1_ID,Flight_1_From,Flight_1_From_IATA,Flight_1_To,Flight_1_To_IATA,Flight_1_start_time(SLST),Flight_1_Arrival_time(SLST),Flight_1_Min,Flight_1_Max,Flight_1_duration_hours,Flight_1_End_Day,Flight_1_Allocated_Members
0,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,1.42,Monday,Mo/MLE/102/CMB
1,Tuesday,UL 173,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Tuesday,Tu/CMB/173/BLR
2,Tuesday,UL 306,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,9.00,Tuesday,Tu/CMB/306/SIN
3,Tuesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Tuesday,Tu/BLR/174/CMB
4,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,4.67,Tuesday,Tu/SIN/307/MLE


In [ ]:
Flight_2_df = final_2

In [ ]:
Flight_2_df.columns=["Flight_2_Start_Day", "Flight_2_ID", "Flight_2_From","Flight_2_From_IATA", "Flight_2_To", "Flight_2_To_IATA", "Flight_2_start_time(SLST)", "Flight_2_Arrival_time(SLST)", "Flight_2_Min",	"Flight_2_Max","Flight_2_duration_hours" , "Flight_2_End_Day"]

In [ ]:
# Perform the join
combined_flights_1 = pd.merge(Flight_1_df, Flight_2_df, left_on='Flight_1_To', right_on='Flight_2_From')

# Display the result
combined_flights_1

,Flight_1_Start_Day,Flight_1_ID,Flight_1_From,Flight_1_From_IATA,Flight_1_To,Flight_1_To_IATA,Flight_1_start_time(SLST),Flight_1_Arrival_time(SLST),Flight_1_Min,Flight_1_Max,...,Flight_2_From,Flight_2_From_IATA,Flight_2_To,Flight_2_To_IATA,Flight_2_start_time(SLST),Flight_2_Arrival_time(SLST),Flight_2_Min,Flight_2_Max,Flight_2_duration_hours,Flight_2_End_Day
0,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Tuesday
1,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,9.00,Tuesday
2,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Wednesday
3,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Thursday
4,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,9.00,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,50,...,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Wednesday
123,Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,50,...,Colombo,CMB,Bengaluru,BLR,01:10:00,02:35:00,3,50,1.42,Thursday
124,Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,50,...,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,9.00,Thursday
125,Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,50,...,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,9.00,Friday


In [ ]:
combined_flights_1.to_csv("combined_flights_1.csv", index=False)

In [ ]:
combined_flights_1.shape

(127, 25)

In [ ]:
import datetime
import pandas as pd

# Function to convert day names (e.g., "Monday") into future dates
def get_date_from_day(day_name, reference_date):
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    if day_name not in weekdays:
        return None

    ref_index = reference_date.weekday()
    day_index = weekdays.index(day_name)

    days_ahead = (day_index - ref_index + 7) % 7
    if days_ahead == 0:
        days_ahead = 7


    return reference_date + datetime.timedelta(days=days_ahead)

In [ ]:
# Load dataset
combination_1 = combined_flights_1

# Reference date (today)
reference_date = pd.Timestamp.now().date()

In [ ]:
reference_date

datetime.date(2025, 4, 28)

In [ ]:
combination_1['Flight 1 Start Date'] = combination_1['Flight_1_Start_Day'].apply(
    lambda day: get_date_from_day(day, reference_date)
)

combination_1['Flight 1 End Date'] = combination_1['Flight_1_End_Day'].apply(
    lambda day: get_date_from_day(day, reference_date)
)

combination_1['Flight 1 End Date'] = combination_1.apply(
    lambda row: row['Flight 1 Start Date'] if row['Flight_1_Start_Day'] == row['Flight_1_End_Day'] else row['Flight 1 End Date'],
    axis=1
)


In [ ]:
import numpy as np
import pandas as pd

# Convert day names to numbers
day_mapping = {
    "Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3,
    "Friday": 4, "Saturday": 5, "Sunday": 6
}

combination_1['Flight_1_End_Day_Num'] = combination_1['Flight_1_End_Day'].map(day_mapping)
combination_1['Flight_2_Start_Day_Num'] = combination_1['Flight_2_Start_Day'].map(day_mapping)
combination_1['Flight_2_End_Day_Num'] = combination_1['Flight_2_End_Day'].map(day_mapping)

combination_1['Flight 1 Start Date'] = pd.to_datetime(combination_1['Flight 1 Start Date'])
combination_1['Flight 1 End Date'] = pd.to_datetime(combination_1['Flight 1 End Date'])


In [ ]:
# # Flight 2 Start Date
# Convert 'Flight 1 End Date' to datetime
combination_1['Flight 1 End Date'] = pd.to_datetime(combination_1['Flight 1 End Date'])

# Ensure it is not categorical
if pd.api.types.is_categorical_dtype(combination_1['Flight 1 End Date']):
    combination_1['Flight 1 End Date'] = combination_1['Flight 1 End Date'].astype('datetime64[ns]')

# Convert to integer if needed
combination_1['Flight_2_Start_Day_Num'] = pd.to_numeric(combination_1['Flight_2_Start_Day_Num'], errors='coerce')

# Get the weekday of Flight 1 End Date (Monday=0, ..., Sunday=6)
combination_1['Flight_1_End_Day_Num'] = combination_1['Flight 1 End Date'].dt.weekday

# Define mask: True if Start Day Num is after or on the same day as End Day Num
mask = combination_1['Flight_2_Start_Day_Num'] >= combination_1['Flight_1_End_Day_Num']

# Case 1: Flight 2 Start is in the same week
combination_1.loc[mask, 'Flight 2 Start Date'] = (
    combination_1['Flight 1 End Date'] +
    pd.to_timedelta(combination_1['Flight_2_Start_Day_Num'] - combination_1['Flight_1_End_Day_Num'], unit='D')
)

# Case 2: Flight 2 Start is in the next week (~mask)
combination_1.loc[~mask, 'Flight 2 Start Date'] = (
    combination_1['Flight 1 End Date'] +
    pd.to_timedelta(7 - (combination_1['Flight_1_End_Day_Num'] - combination_1['Flight_2_Start_Day_Num']), unit='D')
)

<ipython-input-468-6f9fa4942d63>:6: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(combination_1['Flight 1 End Date']):


In [ ]:
# # Flight 2 End Date
# Convert column to datetime
combination_1['Flight 2 Start Date'] = pd.to_datetime(combination_1['Flight 2 Start Date'])

# Ensure it is not categorical
if pd.api.types.is_categorical_dtype(combination_1['Flight 2 Start Date']):
    combination_1['Flight 2 Start Date'] = combination_1['Flight 2 Start Date'].astype('datetime64[ns]')


mask2 = combination_1['Flight_2_End_Day_Num'] >= combination_1['Flight_2_Start_Day_Num']

# Case 1: Flight 2 End is in the same week
combination_1.loc[mask2, 'Flight 2 End Date'] = (
    combination_1['Flight 2 Start Date'] +
    pd.to_timedelta(combination_1['Flight_2_End_Day_Num'] - combination_1['Flight_2_Start_Day_Num'], unit='D')
)

# Case 2: Flight 2 End is in the next week (~mask)
combination_1.loc[~mask2, 'Flight 2 End Date'] = (
    combination_1['Flight 2 Start Date'] +
    pd.to_timedelta(7 - (combination_1['Flight_2_Start_Day_Num'] - combination_1['Flight_2_End_Day_Num']), unit='D')
)

<ipython-input-469-115b9df256b4>:6: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(combination_1['Flight 2 Start Date']):


In [ ]:
# import datetime
# import pandas as pd

# # Function to convert day names (e.g., "Monday") into future dates
# def get_date_from_day(day_name, reference_date):
#     weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

#     if day_name not in weekdays:
#         return None

#     ref_index = reference_date.weekday()
#     day_index = weekdays.index(day_name)

#     days_ahead = (day_index - ref_index + 7) % 7
#     if days_ahead == 0:
#         days_ahead = 7

#     return reference_date + datetime.timedelta(days=days_ahead)

# # Load dataset
# combination_1 = combined_flights_1

# # Reference date (today)
# reference_date = datetime.date.today()

# # --- Flight 1 Start and End Dates ---
# combination_1['Flight 1 Start Date'] = combination_1['Flight_1_Start_Day'].apply(get_date_from_day, reference_date=reference_date)
# combination_1['Flight 1 End Date'] = combination_1['Flight_1_End_Day'].apply(get_date_from_day, reference_date=reference_date)

# # Convert to datetime (fix categorical issue)
# combination_1['Flight 1 Start Date'] = pd.to_datetime(combination_1['Flight 1 Start Date'])
# combination_1['Flight 1 End Date'] = pd.to_datetime(combination_1['Flight 1 End Date'])

# # Ensure Flight 1 End Date is after Start Date
# combination_1.loc[combination_1['Flight 1 End Date'] < combination_1['Flight 1 Start Date'], 'Flight 1 End Date'] += pd.Timedelta(days=7)

# # --- Flight 2 Start and End Dates ---
# combination_1['Flight 2 Start Date'] = combination_1['Flight_2_Start_Day'].apply(get_date_from_day, reference_date=reference_date)
# combination_1['Flight 2 End Date'] = combination_1['Flight_2_End_Day'].apply(get_date_from_day, reference_date=reference_date)

# # Convert to datetime (fix categorical issue)
# combination_1['Flight 2 Start Date'] = pd.to_datetime(combination_1['Flight 2 Start Date'])
# combination_1['Flight 2 End Date'] = pd.to_datetime(combination_1['Flight 2 End Date'])

# # Ensure Flight 2 Start Date is after Flight 1 End Date
# combination_1.loc[combination_1['Flight 2 Start Date'] <= combination_1['Flight 1 End Date'], 'Flight 2 Start Date'] = combination_1['Flight 1 End Date'] + pd.Timedelta(days=1)

# # Ensure Flight 2 End Date is after Flight 2 Start Date
# combination_1.loc[combination_1['Flight 2 End Date'] < combination_1['Flight 2 Start Date'], 'Flight 2 End Date'] += pd.Timedelta(days=7)

# # --- Flight 3 Start and End Dates ---
# combination_1['Flight 3 Start Date'] = combination_1['Flight_3_Start_Day'].apply(get_date_from_day, reference_date=reference_date)
# combination_1['Flight 3 End Date'] = combination_1['Flight_3_End_Day'].apply(get_date_from_day, reference_date=reference_date)

# # Convert to datetime (fix categorical issue)
# combination_1['Flight 3 Start Date'] = pd.to_datetime(combination_1['Flight 3 Start Date'])
# combination_1['Flight 3 End Date'] = pd.to_datetime(combination_1['Flight 3 End Date'])

# # Ensure Flight 3 Start Date is after both Flight 1 and Flight 2 End Dates
# combination_1['Flight 3 Start Date'] = combination_1[['Flight 1 End Date', 'Flight 2 End Date']].max(axis=1) + pd.Timedelta(days=1)

# # Ensure Flight 3 End Date is after Flight 3 Start Date
# combination_1.loc[combination_1['Flight 3 End Date'] < combination_1['Flight 3 Start Date'], 'Flight 3 End Date'] += pd.Timedelta(days=7)

# # Start the first tour accounding to the first day that according to the today date
# combination_1['Flight 1 Start Date'] = combination_1['Flight_1_Start_Day'].apply(get_date_from_day, reference_date=datetime.date.today())
# if(combination_1['Flight_1_Start_Day'] == combination_1['Flight_1_End_Day']).all():
#     combination_1['Flight 1 End Date'] = combination_1['Flight 1 Start Date']
# else:
#   combination_1['Flight 1 End Date'] = combination_1['Flight_1_End_Day'].apply(get_date_from_day, reference_date=datetime.date.today())

# combination_1['Flight 1 End Date'] = combination_1['Flight_1_End_Day'].apply(get_date_from_day, reference_date=datetime.date.today())

# if (combination_1['Flight_1_End_Day'] == combination_1['Flight_2_Start_Day']).all():
#     combination_1['Flight 2 Start Date'] = combination_1['Flight 1 End Date']
# else:
#   combination_1['Flight 2 Start Date'] = combination_1['Flight_2_Start_Day'].apply(get_date_from_day, reference_date=datetime.date.today())


# Convert time columns to proper time format
combination_1['Flight_1_Arrival_time(SLST)'] = pd.to_datetime(
    combination_1['Flight_1_Arrival_time(SLST)'], format="%H:%M:%S", errors='coerce'
).dt.time

combination_1['Flight_2_start_time(SLST)'] = pd.to_datetime(
    combination_1['Flight_2_start_time(SLST)'], format="%H:%M:%S", errors='coerce'
).dt.time

# combination_1_new = combination_1.copy()

# Convert to string before concatenation
combination_1['Flight 1 End Date'] = combination_1['Flight 1 End Date'].astype(str)
combination_1['Flight_1_Arrival_time(SLST)'] = combination_1['Flight_1_Arrival_time(SLST)'].astype(str)

# Create Flight 1 Arrival datetime
combination_1['Flight 1 Arrival datetime'] = combination_1['Flight 1 End Date'] + ' ' + combination_1['Flight_1_Arrival_time(SLST)']
combination_1['Flight 1 Arrival datetime'] = pd.to_datetime(combination_1['Flight 1 Arrival datetime'])

# Ensure both columns are in string format for Flight 2 start datetime
combination_1['Flight 2 Start Date'] = combination_1['Flight 2 Start Date'].astype(str)
combination_1['Flight_2_start_time(SLST)'] = combination_1['Flight_2_start_time(SLST)'].astype(str)

# Combine date and time columns for Flight 2
combination_1['Flight 2 start datetime'] = combination_1['Flight 2 Start Date'] + ' ' + combination_1['Flight_2_start_time(SLST)']
combination_1['Flight 2 start datetime'] = pd.to_datetime(combination_1['Flight 2 start datetime'])
combination_1.to_csv("combination_1_new.csv", index = False)

# combination_1['Flight 2 start datetime']=combination_1_new['Flight 2 start datetime']
# combination_1['Flight 1 Arrival datetime']=combination_1_new['Flight 1 Arrival datetime']

# # Function to combine date and time into full datetime
# def convert_to_datetime(date, time_value):
#     if pd.isna(date) or pd.isna(time_value):
#         return None
#     return datetime.datetime.combine(date, time_value)

# # Apply conversion
# combination_1['Flight 1 Arrival datetime'] = combination_1.apply(
#     lambda row: convert_to_datetime(row['Flight 1 End Date'], row['Flight_1_Arrival_time(SLST)']), axis=1
# )
# combination_1['Flight 2 start datetime'] = combination_1.apply(
#     lambda row: convert_to_datetime(row['Flight 2 Start Date'], row['Flight_2_start_time(SLST)']), axis=1
# )


In [ ]:
filtered_flights = combination_1[
    (
        # Case where the dates are the same
        (
            (combination_1['Flight 2 Start Date'].astype('datetime64[ns]') == combination_1['Flight 1 End Date'].astype('datetime64[ns]')) &
            (combination_1['Flight 1 Arrival datetime'].astype('datetime64[ns]') < combination_1['Flight 2 start datetime'].astype('datetime64[ns]'))
        )
        |
        # Case where the dates are different
        (
            (combination_1['Flight 2 Start Date'].astype('datetime64[ns]') > combination_1['Flight 1 End Date'].astype('datetime64[ns]')) &  # Allow same day
            (combination_1['Flight 2 Start Date'].astype('datetime64[ns]') < (combination_1['Flight 1 End Date'].astype('datetime64[ns]') + pd.Timedelta(days=7))) &  # Less than 7 days apart
            (combination_1['Flight 1 Arrival datetime'].astype('datetime64[ns]') < combination_1['Flight 2 Start Date'].astype('datetime64[ns]'))  # Flight 1 arrives before Flight 2 starts
        )
    )
]



In [ ]:
filtered_flights.shape

(110, 34)

In [ ]:
filtered_flights.to_csv("filtered_flights.csv", index=False)

In [ ]:
# Condition 1: Check for invalid cases where Flight 1 arrival time is greater than Flight 2 start time
invalid_flights = filtered_flights[
    (filtered_flights['Flight 1 End Date'] == filtered_flights['Flight 2 Start Date']) &  # Same day
    (filtered_flights['Flight 1 Arrival datetime'] >= filtered_flights['Flight 2 start datetime'])  # Invalid condition
]

if not invalid_flights.empty:
    print("Invalid flight entries detected where Flight 1 arrival is after or at Flight 2 start time:")
    print(invalid_flights[['Flight 1 Arrival datetime', 'Flight 2 start datetime']])

In [ ]:
# Drop invalid flights from the main DataFrame
filtered_flights = filtered_flights[~filtered_flights.index.isin(invalid_flights.index)]
# Optionally, reset the index if desired
filtered_flights.reset_index(drop=True, inplace=True)

In [ ]:
filtered_flights.shape

(110, 34)

In [ ]:
# Condition 2: Ensure Flight 1 End Date < Flight 2 Start Date
invalid_flights_condition_2 = filtered_flights[
    filtered_flights['Flight 1 End Date'].astype('datetime64[ns]') > (filtered_flights['Flight 2 Start Date'].astype('datetime64[ns]'))
]

if not invalid_flights_condition_2.empty:
    print("Flight 1 End Date < Flight 2 Start Date:")
    print(invalid_flights_condition_2[[ 'Flight 1 End Date', 'Flight 2 Start Date']])

In [ ]:
# Drop invalid flights from the main DataFrame
filtered_flights = filtered_flights[~filtered_flights.index.isin(invalid_flights_condition_2.index)]
# Optionally, reset the index if desired
filtered_flights.reset_index(drop=True, inplace=True)

In [ ]:
# Calculate waiting time
filtered_flights['Waiting Time'] = filtered_flights['Flight 2 start datetime'] - filtered_flights['Flight 1 Arrival datetime']

# Convert waiting time to hours for better readability
filtered_flights['Waiting Time (hours)'] = filtered_flights['Waiting Time'].dt.total_seconds() / 3600

# Apply filtering condition: Keep all rows, but filter when Flight 1 Day == Flight 2 Day
filtered_flights_1 = filtered_flights[
    (filtered_flights['Flight 1 End Date'] != filtered_flights['Flight 2 Start Date']) |  # Keep data where days are different
    (filtered_flights['Flight 1 Arrival datetime'] < filtered_flights['Flight 2 start datetime'])  # Apply condition when days match
]

# Display or save the filtered DataFrame
# filtered_flights.to_csv("filtered_flights.csv", index=False)
filtered_flights_1[[ 'Flight 1 Arrival datetime', 'Flight 2 start datetime', 'Waiting Time (hours)']]
# # Save the filtered results to CSV
# filtered_flights_1.to_csv("filtered_flights_1.csv", index=False)

,Flight 1 Arrival datetime,Flight 2 start datetime,Waiting Time (hours)
0,2025-05-05 11:20:00,2025-05-06 01:10:00,13.833333
1,2025-05-05 11:20:00,2025-05-06 01:10:00,13.833333
2,2025-05-05 11:20:00,2025-05-07 01:10:00,37.833333
3,2025-05-05 11:20:00,2025-05-08 01:10:00,61.833333
4,2025-05-05 11:20:00,2025-05-08 01:10:00,61.833333
...,...,...,...
105,2025-05-05 04:25:00,2025-05-07 01:10:00,44.750000
106,2025-05-05 04:25:00,2025-05-08 01:10:00,68.750000
107,2025-05-05 04:25:00,2025-05-08 01:10:00,68.750000
108,2025-05-05 04:25:00,2025-05-09 01:10:00,92.750000


In [ ]:
# Function to calculate waiting days based on waiting time in hours
def calculate_waiting_days(waiting_time):
    waiting_days = 0
    thresholds = [24, 48, 72, 96, 120, 144, 168]  # Define thresholds in hours

    for threshold in thresholds:
        if waiting_time < threshold:
            break  # Stop if the waiting time is less than the threshold
        waiting_days += 1  # Increment waiting days for each threshold exceeded

    return waiting_days

# Assuming 'Waiting Time' is already in hours in the filtered_flights_1 DataFrame
filtered_flights_1['Waiting Days'] = filtered_flights_1['Waiting Time (hours)'].apply(calculate_waiting_days)

# Drop unnecessary columns (ignoring errors if they don't exist)
filtered_flights_1.drop(columns=['Flight 1 Date', 'Flight 2 Date', 'Flight 1 Arrival datetime', 'Flight 2 start datetime', 'Waiting Time', 'Waiting Time (minutes)','Flight_1_End_Day_Num', 'Flight_2_Start_Day_Num', 'Flight_2_End_Day_Num'], inplace=True, errors='ignore')
# filtered_flights_1["1st_tour_total_flight_hours"] = filtered_flights_1["Flight_1_duration_hours"] + filtered_flights_1["Flight_2_duration_hours"]
# Display or save the updated DataFrame as needed
# filtered_flights_1.to_csv("filtered_flights_1.csv", index=False)

print("Filtered flights with waiting time and waiting days saved successfully!")


Filtered flights with waiting time and waiting days saved successfully!


In [ ]:
filtered_flights_1.columns

Index(['Flight_1_Start_Day', 'Flight_1_ID', 'Flight_1_From',
       'Flight_1_From_IATA', 'Flight_1_To', 'Flight_1_To_IATA',
       'Flight_1_start_time(SLST)', 'Flight_1_Arrival_time(SLST)',
       'Flight_1_Min', 'Flight_1_Max', 'Flight_1_duration_hours',
       'Flight_1_End_Day', 'Flight_1_Allocated_Members', 'Flight_2_Start_Day',
       'Flight_2_ID', 'Flight_2_From', 'Flight_2_From_IATA', 'Flight_2_To',
       'Flight_2_To_IATA', 'Flight_2_start_time(SLST)',
       'Flight_2_Arrival_time(SLST)', 'Flight_2_Min', 'Flight_2_Max',
       'Flight_2_duration_hours', 'Flight_2_End_Day', 'Flight 1 Start Date',
       'Flight 1 End Date', 'Flight 2 Start Date', 'Flight 2 End Date',
       'Waiting Time (hours)', 'Waiting Days'],
      dtype='object')

In [ ]:
# # Assign Allocated_Members dynamically without storing Day_Frequency - Flight 2
filtered_flights_1["Flight_2_Allocated_Members"] = filtered_flights_1["Flight_1_Allocated_Members"].str[:] + "_" + filtered_flights_1["Flight_2_Start_Day"].str[0:2] +"/"+ filtered_flights_1["Flight_2_From_IATA"].str[0:3] + "/" +  filtered_flights_1["Flight_2_ID"].str[-3:] + "/" + filtered_flights_1["Flight_2_To_IATA"].str[0:3]
# final_df

In [ ]:
# # Assign Allocated_Members dynamically without storing Day_Frequency - Flight 2
# filtered_flights_1["Flight_2_Allocated_Members"] = filtered_flights_1["Flight_1_Allocated_Members"].str[0:7] + "_" + filtered_flights_1["Flight_2_To_IATA"].str[0:3] + "_" + filtered_flights_1["Flight_1_Allocated_Members"].str[8:11] + (filtered_flights_1.groupby("Flight_1_Allocated_Members").cumcount() + 1).astype(str)
# # final_df

In [ ]:
filtered_flights_1["Flight_2_Allocated_Members"]

,Flight_2_Allocated_Members
0,Mo/MLE/102/CMB_Tu/CMB/173/BLR
1,Mo/MLE/102/CMB_Tu/CMB/306/SIN
2,Mo/MLE/102/CMB_We/CMB/173/BLR
3,Mo/MLE/102/CMB_Th/CMB/173/BLR
4,Mo/MLE/102/CMB_Th/CMB/306/SIN
...,...
105,Su/LHR/506/CMB_We/CMB/173/BLR
106,Su/LHR/506/CMB_Th/CMB/173/BLR
107,Su/LHR/506/CMB_Th/CMB/306/SIN
108,Su/LHR/506/CMB_Fr/CMB/306/SIN


In [ ]:
unique_count =filtered_flights_1["Flight_2_Allocated_Members"].nunique()

In [ ]:
unique_count

110

In [ ]:
filtered_flights_1.to_csv("filtered_flights_1.csv", index=False)

In [ ]:
# # Count occurrences for 'Flight 1 Allocated_Members'
# counts = filtered_flights_1.groupby("Flight_1_Allocated_Members").cumcount() + 1
# print(counts[9])
# # Convert numbers > 9 to letters (A=10, B=11, C=12, ...)
# def num_to_label(n):
#     return str(n) if n <= 9 else string.ascii_uppercase[n - 10]  # Convert 10 → A, 11 → B, etc.

# # Apply transformation
# filtered_flights_1["Flight_2_Allocated_Members"] = (
#     filtered_flights_1["Flight_2_Allocated_Members"].str[:15] + counts.apply(num_to_label)
# )

In [ ]:
# filtered_flights_1.to_csv("filtered_flights_1.csv", index=False)

In [ ]:
# # Assign Allocated_Members dynamically without storing Day_Frequency - Flight 2
# filtered_flights_1["Flight_2_Allocated_Members"] = filtered_flights_1["Flight_1_Allocated_Members"].str[0] + filtered_flights_1["Flight_1_Allocated_Members"].str[1] + filtered_flights_1["Flight_1_Allocated_Members"].str[2] + (filtered_flights_1.groupby("Flight_1_Allocated_Members").cumcount() + 1).astype(str)


In [ ]:
# filtered_flights_1

In [ ]:
# filtered_flights_1.to_csv("filtered_flights_1.csv", index=False)

In [ ]:
# Define a function to check the condition
def set_tour_type(row):
    if row["Flight_1_From"] == row["Flight_2_To"]:
        return "Return Double Tours (P*2)"
    else:
        return "No Return Double Tours (P3)"

# Apply the function to create the new column
filtered_flights_1["Tour Type"] = filtered_flights_1.apply(set_tour_type, axis=1)

# Move "Tour Type" to the first column
filtered_flights_1.insert(0, "Tour Type", filtered_flights_1.pop("Tour Type"))

# Verify the change
# filtered_flights_1 # Display first few rows to check

In [ ]:
# Separate DataFrames based on Tour Type and reset index
df_return_double = filtered_flights_1[filtered_flights_1["Tour Type"] == "Return Double Tours (P*2)"].reset_index(drop=True)
df_no_return_double = filtered_flights_1[filtered_flights_1["Tour Type"] == "No Return Double Tours (P3)"].reset_index(drop=True)

# Display the shapes of the new DataFrames
print("Return Double Tours:", df_return_double.shape)
print("No Return Double Tours:", df_no_return_double.shape)


Return Double Tours: (28, 33)
No Return Double Tours: (82, 33)


In [ ]:
filtered_flights_1.columns

Index(['Tour Type', 'Flight_1_Start_Day', 'Flight_1_ID', 'Flight_1_From',
       'Flight_1_From_IATA', 'Flight_1_To', 'Flight_1_To_IATA',
       'Flight_1_start_time(SLST)', 'Flight_1_Arrival_time(SLST)',
       'Flight_1_Min', 'Flight_1_Max', 'Flight_1_duration_hours',
       'Flight_1_End_Day', 'Flight_1_Allocated_Members', 'Flight_2_Start_Day',
       'Flight_2_ID', 'Flight_2_From', 'Flight_2_From_IATA', 'Flight_2_To',
       'Flight_2_To_IATA', 'Flight_2_start_time(SLST)',
       'Flight_2_Arrival_time(SLST)', 'Flight_2_Min', 'Flight_2_Max',
       'Flight_2_duration_hours', 'Flight_2_End_Day', 'Flight 1 Start Date',
       'Flight 1 End Date', 'Flight 2 Start Date', 'Flight 2 End Date',
       'Waiting Time (hours)', 'Waiting Days', 'Flight_2_Allocated_Members'],
      dtype='object')

In [ ]:
# Select only the relevant columns
df_selected = filtered_flights_1[['Flight_1_Allocated_Members','Flight 1 Start Date', 'Flight_1_start_time(SLST)', 'Flight 1 End Date', 'Flight_1_Arrival_time(SLST)']]
# Drop duplicates based on column1
result_1 = df_selected.drop_duplicates(subset='Flight_1_Allocated_Members')

# Convert Flight 1 Start Date to datetime
result_1['Flight 1 Start Date'] = pd.to_datetime(result_1['Flight 1 Start Date'])

# Sort by Flight 1 Start Date (now as date)
result_sorted = result_1.sort_values(by='Flight 1 Start Date')
# Save to CSV
result_sorted.to_csv('unique_column1_for_flight1_crew.csv', index=False)


<ipython-input-496-9eae955e6892>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_1['Flight 1 Start Date'] = pd.to_datetime(result_1['Flight 1 Start Date'])


In [ ]:
result_1

,Flight_1_Allocated_Members,Flight 1 Start Date,Flight_1_start_time(SLST),Flight 1 End Date,Flight_1_Arrival_time(SLST)
0,Mo/MLE/102/CMB,2025-05-05,09:55:00,2025-05-05,11:20:00
7,Tu/CMB/173/BLR,2025-04-29,01:10:00,2025-04-29,02:35:00
11,Tu/CMB/306/SIN,2025-04-29,01:10:00,2025-04-29,10:10:00
12,Tu/BLR/174/CMB,2025-04-29,03:35:00,2025-04-29,05:05:00
17,Tu/SIN/307/MLE,2025-04-29,09:45:00,2025-04-29,14:25:00
23,Tu/MLE/102/CMB,2025-04-29,09:55:00,2025-04-29,11:20:00
28,We/CMB/173/BLR,2025-04-30,01:10:00,2025-04-30,02:35:00
32,We/BLR/174/CMB,2025-04-30,03:35:00,2025-04-30,05:05:00
38,We/MLE/102/CMB,2025-04-30,09:55:00,2025-04-30,11:20:00
44,Th/CMB/173/BLR,2025-05-01,01:10:00,2025-05-01,02:35:00


In [ ]:
filtered_flights_1.to_csv("filtered_flights_1.csv", index=False)

In [ ]:
df_no_return_double.to_csv("No_return_double.csv", index=False)

In [ ]:
df_return_double.to_csv("Return_double.csv", index=False)

In [ ]:
df_no_return_double_new = df_no_return_double.drop(columns=["Tour Type"])

In [ ]:
df_no_return_double_new.shape

(82, 32)

In [ ]:
filtered_flights_1.columns

Index(['Tour Type', 'Flight_1_Start_Day', 'Flight_1_ID', 'Flight_1_From',
       'Flight_1_From_IATA', 'Flight_1_To', 'Flight_1_To_IATA',
       'Flight_1_start_time(SLST)', 'Flight_1_Arrival_time(SLST)',
       'Flight_1_Min', 'Flight_1_Max', 'Flight_1_duration_hours',
       'Flight_1_End_Day', 'Flight_1_Allocated_Members', 'Flight_2_Start_Day',
       'Flight_2_ID', 'Flight_2_From', 'Flight_2_From_IATA', 'Flight_2_To',
       'Flight_2_To_IATA', 'Flight_2_start_time(SLST)',
       'Flight_2_Arrival_time(SLST)', 'Flight_2_Min', 'Flight_2_Max',
       'Flight_2_duration_hours', 'Flight_2_End_Day', 'Flight 1 Start Date',
       'Flight 1 End Date', 'Flight 2 Start Date', 'Flight 2 End Date',
       'Waiting Time (hours)', 'Waiting Days', 'Flight_2_Allocated_Members'],
      dtype='object')

In [ ]:
# Separate flights into 1st and 2nd flight categories
# df_no_return_2 = no return double tours drop the type of tour column
df_no_return_2 = df_no_return_double_new[df_no_return_double["Flight_2_Start_Day"].isin(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])] # First flights are sorted by day and start time
Flight_3_df = final_3[final_3["Start_Day"].isin(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])] # Second flights are sorted by arrival time

In [ ]:
Flight_3_df.columns=["Flight_3_Start_Day", "Flight_3_ID", "Flight_3_From","Flight_3_From_IATA", "Flight_3_To","Flight_3_To_IATA", "Flight_3_start_time(SLST)", "Flight_3_Arrival_time(SLST)", "Flight_3_Min",	"Flight_3_Max","Flight_3_duration_hours" , "Flight_3_End_Day"]

In [ ]:
# Perform the join
combined_flights_2 = pd.merge(df_no_return_2, Flight_3_df, left_on='Flight_2_To', right_on='Flight_3_From')

# Display the result
combined_flights_2

,Flight_1_Start_Day,Flight_1_ID,Flight_1_From,Flight_1_From_IATA,Flight_1_To,Flight_1_To_IATA,Flight_1_start_time(SLST),Flight_1_Arrival_time(SLST),Flight_1_Min,Flight_1_Max,...,Flight_3_From,Flight_3_From_IATA,Flight_3_To,Flight_3_To_IATA,Flight_3_start_time(SLST),Flight_3_Arrival_time(SLST),Flight_3_Min,Flight_3_Max,Flight_3_duration_hours,Flight_3_End_Day
0,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Tuesday
1,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Wednesday
2,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Thursday
3,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Saturday
4,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,4.67,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,50,...,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,4.67,Friday
304,Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,50,...,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Tuesday
305,Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,50,...,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Wednesday
306,Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,50,...,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,1.50,Thursday


In [ ]:
combined_flights_2.shape

(308, 44)

In [ ]:
combined_flights_2 = combined_flights_2[combined_flights_2['Flight_1_To'] != combined_flights_2['Flight_3_To']]

In [ ]:
combined_flights_2.to_csv("combined_flights_2.csv", index=False)

In [ ]:
combined_flights_2.shape

(180, 44)

In [ ]:
combined_flights_2.columns

Index(['Flight_1_Start_Day', 'Flight_1_ID', 'Flight_1_From',
       'Flight_1_From_IATA', 'Flight_1_To', 'Flight_1_To_IATA',
       'Flight_1_start_time(SLST)', 'Flight_1_Arrival_time(SLST)',
       'Flight_1_Min', 'Flight_1_Max', 'Flight_1_duration_hours',
       'Flight_1_End_Day', 'Flight_1_Allocated_Members', 'Flight_2_Start_Day',
       'Flight_2_ID', 'Flight_2_From', 'Flight_2_From_IATA', 'Flight_2_To',
       'Flight_2_To_IATA', 'Flight_2_start_time(SLST)',
       'Flight_2_Arrival_time(SLST)', 'Flight_2_Min', 'Flight_2_Max',
       'Flight_2_duration_hours', 'Flight_2_End_Day', 'Flight 1 Start Date',
       'Flight 1 End Date', 'Flight 2 Start Date', 'Flight 2 End Date',
       'Waiting Time (hours)', 'Waiting Days', 'Flight_2_Allocated_Members',
       'Flight_3_Start_Day', 'Flight_3_ID', 'Flight_3_From',
       'Flight_3_From_IATA', 'Flight_3_To', 'Flight_3_To_IATA',
       'Flight_3_start_time(SLST)', 'Flight_3_Arrival_time(SLST)',
       'Flight_3_Min', 'Flight_3_Max', 'Fli

In [ ]:
# # --- Flight 3 Start and End Dates ---
# combination_2['Flight 3 Start Date'] = combination_2['Flight_3_Start_Day'].apply(get_date_from_day, reference_date=reference_date)
# combination_2['Flight 3 End Date'] = combination_1['Flight_3_End_Day'].apply(get_date_from_day, reference_date=reference_date)

# # Convert to datetime (fix categorical issue)
# combination_2['Flight 3 Start Date'] = pd.to_datetime(combination_1['Flight 3 Start Date'])
# combination_1['Flight 3 End Date'] = pd.to_datetime(combination_1['Flight 3 End Date'])

# # Ensure Flight 3 Start Date is after both Flight 1 and Flight 2 End Dates
# combination_1['Flight 3 Start Date'] = combination_1[['Flight 1 End Date', 'Flight 2 End Date']].max(axis=1) + pd.Timedelta(days=1)

# # Ensure Flight 3 End Date is after Flight 3 Start Date
# combination_1.loc[combination_1['Flight 3 End Date'] < combination_1['Flight 3 Start Date'], 'Flight 3 End Date'] += pd.Timedelta(days=7)


In [ ]:
import datetime
import pandas as pd

# Function to convert day names (e.g., "Monday") into future dates
def get_date_from_day(day_name, reference_date):
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    if day_name not in weekdays:
        return None

    ref_index = reference_date.weekday()
    day_index = weekdays.index(day_name)

    days_ahead = (day_index - ref_index + 7) % 7
    if days_ahead == 0:
        days_ahead = 7

    return reference_date + datetime.timedelta(days=days_ahead)

# Load dataset
combination_2 = pd.DataFrame(combined_flights_2)

In [ ]:
import numpy as np
import pandas as pd

# Convert day names to numbers
day_mapping = {
    "Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3,
    "Friday": 4, "Saturday": 5, "Sunday": 6
}

combination_2['Flight_3_End_Day_Num'] = combination_2['Flight_3_End_Day'].map(day_mapping)
combination_2['Flight_3_Start_Day_Num'] = combination_2['Flight_3_Start_Day'].map(day_mapping)
combination_2['Flight_2_Start_Day_Num'] = combination_2['Flight_2_Start_Day'].map(day_mapping)
combination_2['Flight_2_End_Day_Num'] = combination_2['Flight_2_End_Day'].map(day_mapping)

combination_2['Flight 2 Start Date'] = pd.to_datetime(combination_2['Flight 2 Start Date'])
combination_2['Flight 2 End Date'] = pd.to_datetime(combination_2['Flight 2 End Date'])


In [ ]:
# # Flight 3 Start Date
# Convert 'Flight 2 End Date' to datetime
combination_2['Flight 2 End Date'] = pd.to_datetime(combination_2['Flight 2 End Date'])

# Ensure it is not categorical
if pd.api.types.is_categorical_dtype(combination_2['Flight 2 End Date']):
    combination_2['Flight 2 End Date'] = combination_2['Flight 2 End Date'].astype('datetime64[ns]')

# Convert to integer if needed
combination_2['Flight_3_Start_Day_Num'] = pd.to_numeric(combination_2['Flight_3_Start_Day_Num'], errors='coerce')

# Get the weekday of Flight 3 End Date (Monday=0, ..., Sunday=6)
combination_2['Flight_2_End_Day_Num'] = combination_2['Flight 2 End Date'].dt.weekday

# Define mask: True if Start Day Num is after or on the same day as End Day Num
mask3 = combination_2['Flight_3_Start_Day_Num'] >= combination_2['Flight_2_End_Day_Num']

# Case 1: Flight 3 Start is in the same week
combination_2.loc[mask3, 'Flight 3 Start Date'] = (
    combination_2['Flight 2 End Date'] +
    pd.to_timedelta(combination_2['Flight_3_Start_Day_Num'] - combination_2['Flight_2_End_Day_Num'], unit='D')
)

# Case 2: Flight 3 Start is in the next week (~mask)
combination_2.loc[~mask3, 'Flight 3 Start Date'] = (
    combination_2['Flight 2 End Date'] +
    pd.to_timedelta(7 - (combination_2['Flight_2_End_Day_Num'] - combination_2['Flight_3_Start_Day_Num']), unit='D')
)

<ipython-input-515-0cbca34ce0d5>:6: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(combination_2['Flight 2 End Date']):


In [ ]:
# # Flight 3 End Date
# Convert column to datetime
combination_2['Flight 3 Start Date'] = pd.to_datetime(combination_2['Flight 3 Start Date'])

# Ensure it is not categorical
if pd.api.types.is_categorical_dtype(combination_2['Flight 3 Start Date']):
    combination_2['Flight 3 Start Date'] = combination_2['Flight 3 Start Date'].astype('datetime64[ns]')


mask4 = combination_2['Flight_3_End_Day_Num'] >= combination_2['Flight_3_Start_Day_Num']

# Case 1: Flight 3 End is in the same week
combination_2.loc[mask4, 'Flight 3 End Date'] = (
    combination_2['Flight 3 Start Date'] +
    pd.to_timedelta(combination_2['Flight_3_End_Day_Num'] - combination_2['Flight_3_Start_Day_Num'], unit='D')
)

# Case 2: Flight 3 End is in the next week (~mask4)
combination_2.loc[~mask4, 'Flight 3 End Date'] = (
    combination_2['Flight 3 Start Date'] +
    pd.to_timedelta(7 - (combination_2['Flight_3_Start_Day_Num'] - combination_2['Flight_3_End_Day_Num']), unit='D')
)

<ipython-input-516-792dfad36141>:6: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(combination_2['Flight 3 Start Date']):


In [ ]:
# import datetime
# import pandas as pd

# # Function to convert day names (e.g., "Monday") into future dates
# def get_date_from_day(day_name, reference_date):
#     weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

#     if day_name not in weekdays:
#         return None

#     ref_index = reference_date.weekday()
#     day_index = weekdays.index(day_name)

#     days_ahead = (day_index - ref_index + 7) % 7
#     if days_ahead == 0:
#         days_ahead = 7

#     return reference_date + datetime.timedelta(days=days_ahead)

# Load dataset
# combination_2 = combined_flights_2

# # Convert 'Flight 2 End Day' to actual dates
# combination_2['Flight 2 End Date'] = combination_2['Flight_2_End_Day'].apply(get_date_from_day, reference_date=datetime.date.today())

# combination_2
# if (combination_2['Flight_2_End_Day'] == combination_2['Flight_3_Start_Day']).all():
#     combination_2['Flight 3 Start Date'] = combination_2['Flight 2 End Date']
# else:
#   combination_2['Flight 3 Start Date'] = combination_2['Flight_3_Start_Day'].apply(get_date_from_day, reference_date=datetime.date.today())

# # --- Flight 3 Start and End Dates ---
# combination_2['Flight 3 Start Date'] = combination_2['Flight_3_Start_Day'].apply(get_date_from_day, reference_date=reference_date)
# combination_2['Flight 3 End Date'] = combination_2['Flight_3_End_Day'].apply(get_date_from_day, reference_date=reference_date)

# # Convert to datetime (fix categorical issue)
# combination_2['Flight 3 Start Date'] = pd.to_datetime(combination_2['Flight 3 Start Date'])
# combination_2['Flight 3 End Date'] = pd.to_datetime(combination_2['Flight 3 End Date'])

# # Ensure Flight 3 Start Date is after both Flight 1 and Flight 2 End Dates
# combination_2['Flight 3 Start Date'] = combination_2[['Flight 1 End Date', 'Flight 2 End Date']].max(axis=1) + pd.Timedelta(days=1)

# if (combination_2['Flight_3_Start_Day'] == combination_2['Flight_3_End_Day']).all():
#     combination_2['Flight 3 End Date'] = combination_2['Flight 3 Start Date']
# else:
#   # Ensure Flight 3 End Date is after Flight 3 Start Date
#   combination_2.loc[combination_2['Flight 3 End Date'] < combination_2['Flight 3 Start Date'], 'Flight 3 End Date'] += pd.Timedelta(days=7)

# Convert time columns to proper time format
combination_2['Flight_2_Arrival_time(SLST)'] = pd.to_datetime(
    combination_2['Flight_2_Arrival_time(SLST)'], format="%H:%M:%S", errors='coerce'
).dt.time

combination_2['Flight_3_start_time(SLST)'] = pd.to_datetime(
    combination_2['Flight_3_start_time(SLST)'], format="%H:%M:%S", errors='coerce'
).dt.time

combination_2_new = combination_2.copy()

# Convert to string before concatenation
combination_2_new['Flight 2 End Date'] = combination_2_new['Flight 2 End Date'].astype(str)
combination_2_new['Flight_2_Arrival_time(SLST)'] = combination_2_new['Flight_2_Arrival_time(SLST)'].astype(str)

# Create Flight 2 Arrival datetime
combination_2_new['Flight 2 Arrival datetime'] = combination_2_new['Flight 2 End Date'] + ' ' + combination_2_new['Flight_2_Arrival_time(SLST)']
combination_2_new['Flight 2 Arrival datetime'] = pd.to_datetime(combination_2_new['Flight 2 Arrival datetime'])

# Ensure both columns are in string format for Flight 3 start datetime
combination_2_new['Flight 3 Start Date'] = combination_2_new['Flight 3 Start Date'].astype(str)
combination_2_new['Flight_3_start_time(SLST)'] = combination_2_new['Flight_3_start_time(SLST)'].astype(str)

# Combine date and time columns for Flight 3
combination_2_new['Flight 3 start datetime'] = combination_2_new['Flight 3 Start Date'] + ' ' + combination_2_new['Flight_3_start_time(SLST)']
combination_2_new['Flight 3 start datetime'] = pd.to_datetime(combination_2_new['Flight 3 start datetime'])

combination_2['Flight 3 start datetime']=combination_2_new['Flight 3 start datetime']
combination_2['Flight 2 Arrival datetime']=combination_2_new['Flight 2 Arrival datetime']

combination_2.to_csv("combination_2_new.csv", index = False)

In [ ]:
# Ensure datetime format
combination_2['Flight 1 Start Date'] = pd.to_datetime(combination_2['Flight 1 Start Date']).astype('datetime64[ns]')
combination_2['Flight 3 End Date'] = pd.to_datetime(combination_2['Flight 3 End Date']).astype('datetime64[ns]')

# Calculate the difference in days
combination_2['Day Gap'] = (combination_2['Flight 3 End Date'] - combination_2['Flight 1 Start Date']).dt.days

# Filter rows where the gap is outside the range [0, 6]
invalid_flights_condition_3 = combination_2[
    (combination_2['Day Gap'] < 0) | (combination_2['Day Gap'] > 6)
]

# Display invalid cases
if not invalid_flights_condition_3.empty:
    print("Invalid flights where Flight 3 End Date is not within 0 to 6 days from Flight 1 Start Date:")
    print(invalid_flights_condition_3[['Flight 1 Start Date', 'Flight 3 End Date', 'Day Gap']])


Invalid flights where Flight 3 End Date is not within 0 to 6 days from Flight 1 Start Date:
    Flight 1 Start Date Flight 3 End Date  Day Gap
14           2025-05-05        2025-05-13        8
16           2025-05-05        2025-05-13        8
23           2025-04-29        2025-05-06        7
24           2025-04-29        2025-05-07        8
25           2025-04-29        2025-05-08        9
..                  ...               ...      ...
258          2025-05-03        2025-05-13       10
278          2025-05-04        2025-05-13        9
280          2025-05-04        2025-05-13        9
300          2025-05-04        2025-05-13        9
302          2025-05-04        2025-05-13        9

[82 rows x 3 columns]


In [ ]:
combination_2.shape

(180, 53)

In [ ]:
# Drop invalid flights from the main DataFrame
combination_2 = combination_2[~combination_2.index.isin(invalid_flights_condition_3.index)]
# Optionally, reset the index if desired
combination_2.reset_index(drop=True, inplace=True)

In [ ]:
combination_2.shape

(98, 53)

In [ ]:
# Condition 2: Ensure Flight 3 End Date is within 7 days of Flight 1 Start Date
invalid_flights_condition_2 = combination_2[
    combination_2['Flight 3 End Date'] > (combination_2['Flight 1 Start Date'] + pd.Timedelta(days=6))
]

if not invalid_flights_condition_2.empty:
    print("Invalid flight entries detected where Flight 3 End Date exceeds 7 days from Flight 1 Start Date:")
    print(invalid_flights_condition_2[[ 'Flight 1 Start Date', 'Flight 3 End Date']])

# Drop invalid flights from the main DataFrame for Condition 2
combination_2 = combination_2[~combination_2.index.isin(invalid_flights_condition_2.index)]

# Optionally, reset the index if desired
combination_2.reset_index(drop=True, inplace=True)

filtered_flights_2 = combination_2
# Drop unnecessary columns (ignoring errors if they don't exist)
# filtered_flights_2.drop(columns=["Flight 3 Date", "Flight 3 start datetime", ], inplace=True, errors='ignore')

# Verify data types to ensure proper filtering
# print("Filtered data types:")
# print(filtered_flights_2.dtypes)

In [ ]:
filtered_flights_2.shape

(98, 53)

In [ ]:
filtered_flights_2.columns

Index(['Flight_1_Start_Day', 'Flight_1_ID', 'Flight_1_From',
       'Flight_1_From_IATA', 'Flight_1_To', 'Flight_1_To_IATA',
       'Flight_1_start_time(SLST)', 'Flight_1_Arrival_time(SLST)',
       'Flight_1_Min', 'Flight_1_Max', 'Flight_1_duration_hours',
       'Flight_1_End_Day', 'Flight_1_Allocated_Members', 'Flight_2_Start_Day',
       'Flight_2_ID', 'Flight_2_From', 'Flight_2_From_IATA', 'Flight_2_To',
       'Flight_2_To_IATA', 'Flight_2_start_time(SLST)',
       'Flight_2_Arrival_time(SLST)', 'Flight_2_Min', 'Flight_2_Max',
       'Flight_2_duration_hours', 'Flight_2_End_Day', 'Flight 1 Start Date',
       'Flight 1 End Date', 'Flight 2 Start Date', 'Flight 2 End Date',
       'Waiting Time (hours)', 'Waiting Days', 'Flight_2_Allocated_Members',
       'Flight_3_Start_Day', 'Flight_3_ID', 'Flight_3_From',
       'Flight_3_From_IATA', 'Flight_3_To', 'Flight_3_To_IATA',
       'Flight_3_start_time(SLST)', 'Flight_3_Arrival_time(SLST)',
       'Flight_3_Min', 'Flight_3_Max', 'Fli

In [ ]:
filtered_flights_2 = filtered_flights_2[
    (
        # Case where the dates are the same
        (
            (filtered_flights_2['Flight 3 Start Date'].astype('datetime64[ns]') == filtered_flights_2['Flight 2 End Date'].astype('datetime64[ns]')) &
            (filtered_flights_2['Flight 2 Arrival datetime'].astype('datetime64[ns]') < filtered_flights_2['Flight 3 start datetime'].astype('datetime64[ns]'))
        )
        |
        # Case where the dates are different
        (
            (filtered_flights_2['Flight 3 Start Date'].astype('datetime64[ns]') > filtered_flights_2['Flight 2 End Date'].astype('datetime64[ns]')) &  # Allow same day
            (filtered_flights_2['Flight 3 Start Date'].astype('datetime64[ns]') < (filtered_flights_2['Flight 2 End Date'].astype('datetime64[ns]') + pd.Timedelta(days=7))) &  # Less than 7 days apart
            (filtered_flights_2['Flight 2 Arrival datetime'].astype('datetime64[ns]') < filtered_flights_2['Flight 3 Start Date'].astype('datetime64[ns]'))  # Flight 2 arrives before Flight 3 starts
        )
    )
]
# Optionally, reset the index if desired
filtered_flights_2.reset_index(drop=True, inplace=True)

In [ ]:
filtered_flights_2

,Flight_1_Start_Day,Flight_1_ID,Flight_1_From,Flight_1_From_IATA,Flight_1_To,Flight_1_To_IATA,Flight_1_start_time(SLST),Flight_1_Arrival_time(SLST),Flight_1_Min,Flight_1_Max,...,Flight_3_End_Day,Flight_3_End_Day_Num,Flight_3_Start_Day_Num,Flight_2_Start_Day_Num,Flight_2_End_Day_Num,Flight 3 Start Date,Flight 3 End Date,Flight 3 start datetime,Flight 2 Arrival datetime,Day Gap
0,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Friday,4,4,1,1,2025-05-09,2025-05-09,2025-05-09 09:45:00,2025-05-06 10:10:00,4
1,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Friday,4,4,3,3,2025-05-09,2025-05-09,2025-05-09 09:45:00,2025-05-08 10:10:00,4
2,Tuesday,UL 306,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,...,Monday,0,0,4,4,2025-05-05,2025-05-05,2025-05-05 09:55:00,2025-05-02 14:25:00,6
3,Tuesday,UL 306,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,...,Saturday,5,5,4,4,2025-05-03,2025-05-03,2025-05-03 09:55:00,2025-05-02 14:25:00,4
4,Tuesday,UL 306,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,...,Sunday,6,6,4,4,2025-05-04,2025-05-04,2025-05-04 09:55:00,2025-05-02 14:25:00,5
5,Tuesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,...,Friday,4,4,3,3,2025-05-02,2025-05-02,2025-05-02 09:45:00,2025-05-01 10:10:00,3
6,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,Thursday,3,3,2,2,2025-05-01,2025-05-01,2025-05-01 01:10:00,2025-04-30 11:20:00,2
7,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,Thursday,3,3,2,2,2025-05-01,2025-05-01,2025-05-01 01:10:00,2025-04-30 11:20:00,2
8,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,Friday,4,4,2,2,2025-05-02,2025-05-02,2025-05-02 01:10:00,2025-04-30 11:20:00,3
9,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,Saturday,5,5,2,2,2025-05-03,2025-05-03,2025-05-03 01:10:00,2025-04-30 11:20:00,4


In [ ]:
# # Calculate waiting time
# filtered_flights['Waiting Time'] = filtered_flights['Flight 2 start datetime'] - filtered_flights['Flight 1 Arrival datetime']

# # Convert waiting time to hours for better readability
# filtered_flights['Waiting Time (hours)'] = filtered_flights['Waiting Time'].dt.total_seconds() / 3600

# # Apply filtering condition: Keep all rows, but filter when Flight 1 Day == Flight 2 Day
# filtered_flights_1 = filtered_flights[
#     (filtered_flights['Flight 1 End Date'] != filtered_flights['Flight 2 Start Date']) |  # Keep data where days are different
#     (filtered_flights['Flight 1 Arrival datetime'] < filtered_flights['Flight 2 start datetime'])  # Apply condition when days match
# ]

# # Display or save the filtered DataFrame
# # filtered_flights.to_csv("filtered_flights.csv", index=False)
# filtered_flights_1[[ 'Flight 1 Arrival datetime', 'Flight 2 start datetime', 'Waiting Time (hours)']]
# # # Save the filtered results to CSV
# # filtered_flights_1.to_csv("filtered_flights_1.csv", index=False)

In [ ]:
filtered_flights_2.shape

(60, 53)

In [ ]:
filtered_flights_2.to_csv("filtered_flights_2.csv", index=False)

In [ ]:
# Calculate waiting time
filtered_flights_2['Waiting Time'] = filtered_flights_2['Flight 3 start datetime'] - filtered_flights_2['Flight 2 Arrival datetime']

# Convert waiting time to hours for better readability
filtered_flights_2['Waiting Time (hours)_2'] = filtered_flights_2['Waiting Time'].dt.total_seconds() / 3600

# Apply filtering condition: Keep all rows, but filter when Flight 1 Day == Flight 2 Day
filtered_flights_2 = filtered_flights_2[
    (filtered_flights_2['Flight 2 End Date'] != filtered_flights_2['Flight 3 Start Date']) |  # Keep data where days are different
    (filtered_flights_2['Flight 2 Arrival datetime'] < filtered_flights_2['Flight 3 start datetime'])  # Apply condition when days match
]

# Display or save the filtered DataFrame
# filtered_flights.to_csv("filtered_flights.csv", index=False)
filtered_flights_2[['Flight 2 Arrival datetime', 'Flight 3 start datetime', 'Waiting Time (hours)_2']]
# # Save the filtered results to CSV
# filtered_flights_1.to_csv("filtered_flights_1.csv", index=False)

<ipython-input-530-e1e56836eae3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_flights_2['Waiting Time'] = filtered_flights_2['Flight 3 start datetime'] - filtered_flights_2['Flight 2 Arrival datetime']
<ipython-input-530-e1e56836eae3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_flights_2['Waiting Time (hours)_2'] = filtered_flights_2['Waiting Time'].dt.total_seconds() / 3600


,Flight 2 Arrival datetime,Flight 3 start datetime,Waiting Time (hours)_2
0,2025-05-06 10:10:00,2025-05-09 09:45:00,71.583333
1,2025-05-08 10:10:00,2025-05-09 09:45:00,23.583333
2,2025-05-02 14:25:00,2025-05-05 09:55:00,67.500000
3,2025-05-02 14:25:00,2025-05-03 09:55:00,19.500000
4,2025-05-02 14:25:00,2025-05-04 09:55:00,43.500000
5,2025-05-01 10:10:00,2025-05-02 09:45:00,23.583333
6,2025-04-30 11:20:00,2025-05-01 01:10:00,13.833333
7,2025-04-30 11:20:00,2025-05-01 01:10:00,13.833333
8,2025-04-30 11:20:00,2025-05-02 01:10:00,37.833333
9,2025-04-30 11:20:00,2025-05-03 01:10:00,61.833333


In [ ]:
# Function to calculate waiting days based on waiting time in hours
def calculate_waiting_days(waiting_time):
    waiting_days = 0
    thresholds = [24, 48, 72, 96, 120, 144, 168]  # Define thresholds in hours

    for threshold in thresholds:
        if waiting_time < threshold:
            break  # Stop if the waiting time is less than the threshold
        waiting_days += 1  # Increment waiting days for each threshold exceeded

    return waiting_days

# Assuming 'Waiting Time' is already in hours in the filtered_flights_2 DataFrame
filtered_flights_2['Waiting_Days_2'] = filtered_flights_2['Waiting Time (hours)_2'].apply(calculate_waiting_days)

# Drop unnecessary columns (ignoring errors if they don't exist)
filtered_flights_2.drop(columns=['Waiting Time', 'Flight 2 Arrival datetime', 'Flight 3 start datetime', 'Flight 2 start datetime'], inplace=True, errors='ignore')
# filtered_flights_2["1st_tour_total_flight_hours"] = filtered_flights_2["Flight_1_duration_hours"] + filtered_flights_1["Flight_2_duration_hours"]
# Display or save the updated DataFrame as needed
# filtered_flights_2.to_csv("filtered_flights_2.csv", index=False)

print("Filtered flights with waiting time and waiting days saved successfully!")


Filtered flights with waiting time and waiting days saved successfully!


In [ ]:
filtered_flights_2.to_csv("filtered_flights_2.csv", index = False)

In [ ]:
filtered_flights_3 = filtered_flights_2.copy(deep=True)

In [ ]:
filtered_flights_3.columns

Index(['Flight_1_Start_Day', 'Flight_1_ID', 'Flight_1_From',
       'Flight_1_From_IATA', 'Flight_1_To', 'Flight_1_To_IATA',
       'Flight_1_start_time(SLST)', 'Flight_1_Arrival_time(SLST)',
       'Flight_1_Min', 'Flight_1_Max', 'Flight_1_duration_hours',
       'Flight_1_End_Day', 'Flight_1_Allocated_Members', 'Flight_2_Start_Day',
       'Flight_2_ID', 'Flight_2_From', 'Flight_2_From_IATA', 'Flight_2_To',
       'Flight_2_To_IATA', 'Flight_2_start_time(SLST)',
       'Flight_2_Arrival_time(SLST)', 'Flight_2_Min', 'Flight_2_Max',
       'Flight_2_duration_hours', 'Flight_2_End_Day', 'Flight 1 Start Date',
       'Flight 1 End Date', 'Flight 2 Start Date', 'Flight 2 End Date',
       'Waiting Time (hours)', 'Waiting Days', 'Flight_2_Allocated_Members',
       'Flight_3_Start_Day', 'Flight_3_ID', 'Flight_3_From',
       'Flight_3_From_IATA', 'Flight_3_To', 'Flight_3_To_IATA',
       'Flight_3_start_time(SLST)', 'Flight_3_Arrival_time(SLST)',
       'Flight_3_Min', 'Flight_3_Max', 'Fli

In [ ]:
filtered_flights_3.shape

(60, 53)

In [ ]:
filtered_flights_3 = filtered_flights_3.drop(
    ['Flight_3_Start_Day', 'Flight_3_ID', 'Flight_3_From',
       'Flight_3_From_IATA', 'Flight_3_To', 'Flight_3_To_IATA',
       'Flight_3_start_time(SLST)', 'Flight_3_Arrival_time(SLST)',
       'Flight_3_Min', 'Flight_3_Max', 'Flight_3_duration_hours',
       'Flight_3_End_Day', 'Flight_3_End_Day_Num', 'Flight_3_Start_Day_Num',
       'Flight 3 Start Date', 'Flight 3 End Date', 'Day Gap',
       'Waiting Time (hours)_2', 'Waiting_Days_2'],
    axis=1  # Drop columns, not rows
)


In [ ]:
filtered_flights_3.shape

(60, 34)

In [ ]:
colums_2=filtered_flights_3.columns

In [ ]:
# Get unique rows based on column 'A' (keeping the first occurrence)
filtered_flights_3 = filtered_flights_3.drop_duplicates(subset=['Flight_2_Allocated_Members'])

# Optionally, reset the index if desired
filtered_flights_3.reset_index(drop=True, inplace=True)
filtered_flights_3[colums_2]

,Flight_1_Start_Day,Flight_1_ID,Flight_1_From,Flight_1_From_IATA,Flight_1_To,Flight_1_To_IATA,Flight_1_start_time(SLST),Flight_1_Arrival_time(SLST),Flight_1_Min,Flight_1_Max,...,Flight_2_End_Day,Flight 1 Start Date,Flight 1 End Date,Flight 2 Start Date,Flight 2 End Date,Waiting Time (hours),Waiting Days,Flight_2_Allocated_Members,Flight_2_Start_Day_Num,Flight_2_End_Day_Num
0,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Tuesday,2025-05-05,2025-05-05,2025-05-06,2025-05-06,13.833333,0,Mo/MLE/102/CMB_Tu/CMB/306/SIN,1,1
1,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Thursday,2025-05-05,2025-05-05,2025-05-08,2025-05-08,61.833333,2,Mo/MLE/102/CMB_Th/CMB/306/SIN,3,3
2,Tuesday,UL 306,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,...,Friday,2025-04-29,2025-04-29,2025-05-02,2025-05-02,71.583333,2,Tu/CMB/306/SIN_Fr/SIN/307/MLE,4,4
3,Tuesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,...,Thursday,2025-04-29,2025-04-29,2025-05-01,2025-05-01,44.083333,1,Tu/BLR/174/CMB_Th/CMB/306/SIN,3,3
4,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,Wednesday,2025-04-29,2025-04-29,2025-04-30,2025-04-30,19.500000,0,Tu/SIN/307/MLE_We/MLE/102/CMB,2,2
5,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,Thursday,2025-04-29,2025-04-29,2025-05-01,2025-05-01,43.500000,1,Tu/SIN/307/MLE_Th/MLE/102/CMB,3,3
6,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,Friday,2025-04-29,2025-04-29,2025-05-02,2025-05-02,67.500000,2,Tu/SIN/307/MLE_Fr/MLE/102/CMB,4,4
7,Tuesday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,Thursday,2025-04-29,2025-04-29,2025-05-01,2025-05-01,37.833333,1,Tu/MLE/102/CMB_Th/CMB/306/SIN,3,3
8,Wednesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,...,Thursday,2025-04-30,2025-04-30,2025-05-01,2025-05-01,20.083333,0,We/BLR/174/CMB_Th/CMB/306/SIN,3,3
9,Wednesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,...,Friday,2025-04-30,2025-04-30,2025-05-02,2025-05-02,44.083333,1,We/BLR/174/CMB_Fr/CMB/306/SIN,4,4


In [ ]:
filtered_flights_3.shape

(32, 34)

In [ ]:
filtered_flights_3['Flight_3_Start_Day'] = np.nan
filtered_flights_3['Flight_3_ID'] = np.nan
filtered_flights_3['Flight_3_From'] = np.nan
filtered_flights_3['Flight_3_To'] = np.nan
filtered_flights_3['Flight_3_start_time(SLST)'] = np.nan
filtered_flights_3['Flight_3_Arrival_time(SLST)'] = np.nan
filtered_flights_3['Flight_3_Min'] = np.nan
filtered_flights_3['Flight_3_Max'] = np.nan
filtered_flights_3['Flight_3_duration_hours'] = np.nan
filtered_flights_3['Flight_3_End_Day_Num'] = np.nan
filtered_flights_3['Flight_3_Start_Day_Num'] = np.nan
filtered_flights_3['Flight 3 Start Date'] = np.nan
filtered_flights_3['Flight 3 End Date'] = np.nan
filtered_flights_3['Flight 3 start datetime'] = np.nan
filtered_flights_3['Flight 2 Arrival datetime'] = np.nan
filtered_flights_3['Day Gap'] = np.nan
filtered_flights_3['Waiting Time'] = np.nan
filtered_flights_3['Waiting Time (hours)_2'] = np.nan

In [ ]:
filtered_flights_3

,Flight_1_Start_Day,Flight_1_ID,Flight_1_From,Flight_1_From_IATA,Flight_1_To,Flight_1_To_IATA,Flight_1_start_time(SLST),Flight_1_Arrival_time(SLST),Flight_1_Min,Flight_1_Max,...,Flight_3_duration_hours,Flight_3_End_Day_Num,Flight_3_Start_Day_Num,Flight 3 Start Date,Flight 3 End Date,Flight 3 start datetime,Flight 2 Arrival datetime,Day Gap,Waiting Time,Waiting Time (hours)_2
0,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tuesday,UL 306,Colombo,CMB,Singapore,SIN,01:10:00,10:10:00,3,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tuesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Tuesday,UL 307,Singapore,SIN,Male,MLE,09:45:00,14:25:00,6,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Tuesday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Wednesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Wednesday,UL 174,Bengaluru,BLR,Colombo,CMB,03:35:00,05:05:00,3,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Append rows from df2 to df1
df_combined = pd.concat([filtered_flights_3, filtered_flights_2], ignore_index=True)

<ipython-input-543-98bd215c1e9d>:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([filtered_flights_3, filtered_flights_2], ignore_index=True)


In [ ]:
filtered_flights_2.shape

(60, 53)

In [ ]:
df_combined.shape

(92, 56)

In [ ]:
df_combined.dtypes

,0
Flight_1_Start_Day,category
Flight_1_ID,object
Flight_1_From,object
Flight_1_From_IATA,object
Flight_1_To,object
Flight_1_To_IATA,object
Flight_1_start_time(SLST),object
Flight_1_Arrival_time(SLST),object
Flight_1_Min,int64
Flight_1_Max,int64


In [ ]:
df_combined = df_combined.drop(["Flight_3_End_Day_Num", "Flight_3_Start_Day_Num",'Flight 3 start datetime',
       'Flight 2 Arrival datetime','Flight_2_Start_Day_Num', 'Flight_2_End_Day_Num' ], axis=1)
# # Group by 'Category' and 'Type', then sum the 'Value' column
# grouped_df = df_combined.groupby(['Flight_1_Start_Day', 'Flight_1_ID']).sum(numeric_only=True).reset_index()

In [ ]:
import numpy as np

df_combined["Flight_3_Allocated_Members"] = np.where(
    df_combined["Flight_3_Start_Day"].notna(),
    df_combined["Flight_2_Allocated_Members"].astype(str) + "_" +
    df_combined["Flight_3_Start_Day"].str[:2] + "/" +
    df_combined["Flight_3_From_IATA"].str[:3] + "/" +
    df_combined["Flight_3_ID"].str[-3:] + "/" +
    df_combined["Flight_3_To_IATA"].str[:3],
    df_combined["Flight_2_Allocated_Members"].astype(str) + "_" + "NaN"
)


In [ ]:
df_combined["Flight_3_Allocated_Members"]

,Flight_3_Allocated_Members
0,Mo/MLE/102/CMB_Tu/CMB/306/SIN_NaN
1,Mo/MLE/102/CMB_Th/CMB/306/SIN_NaN
2,Tu/CMB/306/SIN_Fr/SIN/307/MLE_NaN
3,Tu/BLR/174/CMB_Th/CMB/306/SIN_NaN
4,Tu/SIN/307/MLE_We/MLE/102/CMB_NaN
...,...
87,Sa/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE
88,Su/MLE/102/CMB_Tu/CMB/306/SIN_Fr/SIN/307/MLE
89,Su/MLE/102/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE
90,Su/LHR/506/CMB_Tu/CMB/306/SIN_Fr/SIN/307/MLE


In [ ]:
unique_count =df_combined["Flight_3_Allocated_Members"].nunique()

In [ ]:
unique_count

92

In [ ]:
df_combined.to_csv("df_combined.csv", index=False)

In [ ]:
# # Assign Allocated_Members dynamically without storing Day_Frequency - Flight 3
# df_combined["Flight_3_Allocated_Members"] = df_combined["Flight_2_Allocated_Members"].str[0:4] + (df_combined.groupby("Flight_2_Allocated_Members").cumcount() + 1).astype(str)

In [ ]:
# # Count occurrences for 'Flight 1 Allocated_Members'
# counts = df_combined.groupby("Flight_3_Allocated_Members").cumcount() + 1

# # Convert numbers > 9 to letters (A=10, B=11, C=12, ...)
# def num_to_label(n):
#     if n <= 9:
#         return str(n)
#     else:
#         return string.ascii_uppercase[n - 10]  # Convert 10 → A, 11 → B, etc.

# # Create 'Flight 2 Allocated' column
# df_combined["Flight_3_Allocated_Members"] = (
#     df_combined["Flight_2_Allocated_Members"].str[0:4] + counts.apply(num_to_label).astype(str)
# )

# df_combined

In [ ]:
# Save the final filtered dataset
df_combined.to_csv("filtered_flights_2.csv", index=False)

In [ ]:
# Define a function to check the condition
def set_tour_type(row):
    if row["Flight_1_From"] == row["Flight_3_To"]:
        return "Return Triple Tours (C3)"
    elif pd.isna(row["Flight_3_From"]) and row["Flight_1_From"] != row["Flight_2_To"]:
        return "No Return double Tours (P3)"
    else:
        return "No Return Triple Tours (P4)"

# Apply the function to create the new column
df_combined["Tour Type"] = df_combined.apply(set_tour_type, axis=1)

# Move "Tour Type" to the first column
tour_type_column = df_combined.pop("Tour Type")  # Remove the column
df_combined.insert(0, "Tour Type", tour_type_column)  # Insert it at the first position

print("Tour Type column successfully moved to the first position!")


Tour Type column successfully moved to the first position!


In [ ]:
df_combined.to_csv("filtered_flights_2.csv", index=False)

In [ ]:
# # Separate DataFrames based on Tour Type and reset index
# df_return_triple = filtered_flights_2[filtered_flights_2["Tour Type"] == "Return Triple Tours (C3)"].reset_index(drop=True)
# df_no_return_triple = filtered_flights_2[filtered_flights_2["Tour Type"] == "No Return Triple Tours (P4)"].reset_index(drop=True)

# # Display the shapes of the new DataFrames
# print("Return Triple Tours:", df_return_triple.shape)
# print("No Return Triple Tours:", df_no_return_triple.shape)

In [ ]:
# df_return_triple.to_csv("Return_triple.csv", index=False)
# df_no_return_triple.to_csv("No_return_triple.csv", index=False)

In [ ]:
df_combined.columns

Index(['Tour Type', 'Flight_1_Start_Day', 'Flight_1_ID', 'Flight_1_From',
       'Flight_1_From_IATA', 'Flight_1_To', 'Flight_1_To_IATA',
       'Flight_1_start_time(SLST)', 'Flight_1_Arrival_time(SLST)',
       'Flight_1_Min', 'Flight_1_Max', 'Flight_1_duration_hours',
       'Flight_1_End_Day', 'Flight_1_Allocated_Members', 'Flight_2_Start_Day',
       'Flight_2_ID', 'Flight_2_From', 'Flight_2_From_IATA', 'Flight_2_To',
       'Flight_2_To_IATA', 'Flight_2_start_time(SLST)',
       'Flight_2_Arrival_time(SLST)', 'Flight_2_Min', 'Flight_2_Max',
       'Flight_2_duration_hours', 'Flight_2_End_Day', 'Flight 1 Start Date',
       'Flight 1 End Date', 'Flight 2 Start Date', 'Flight 2 End Date',
       'Waiting Time (hours)', 'Waiting Days', 'Flight_2_Allocated_Members',
       'Flight_3_Start_Day', 'Flight_3_ID', 'Flight_3_From', 'Flight_3_To',
       'Flight_3_start_time(SLST)', 'Flight_3_Arrival_time(SLST)',
       'Flight_3_Min', 'Flight_3_Max', 'Flight_3_duration_hours',
       'Fligh

In [ ]:
filtered_flights_1.columns

Index(['Tour Type', 'Flight_1_Start_Day', 'Flight_1_ID', 'Flight_1_From',
       'Flight_1_From_IATA', 'Flight_1_To', 'Flight_1_To_IATA',
       'Flight_1_start_time(SLST)', 'Flight_1_Arrival_time(SLST)',
       'Flight_1_Min', 'Flight_1_Max', 'Flight_1_duration_hours',
       'Flight_1_End_Day', 'Flight_1_Allocated_Members', 'Flight_2_Start_Day',
       'Flight_2_ID', 'Flight_2_From', 'Flight_2_From_IATA', 'Flight_2_To',
       'Flight_2_To_IATA', 'Flight_2_start_time(SLST)',
       'Flight_2_Arrival_time(SLST)', 'Flight_2_Min', 'Flight_2_Max',
       'Flight_2_duration_hours', 'Flight_2_End_Day', 'Flight 1 Start Date',
       'Flight 1 End Date', 'Flight 2 Start Date', 'Flight 2 End Date',
       'Waiting Time (hours)', 'Waiting Days', 'Flight_2_Allocated_Members'],
      dtype='object')

In [ ]:
filtered_flights_1

,Tour Type,Flight_1_Start_Day,Flight_1_ID,Flight_1_From,Flight_1_From_IATA,Flight_1_To,Flight_1_To_IATA,Flight_1_start_time(SLST),Flight_1_Arrival_time(SLST),Flight_1_Min,...,Flight_2_Max,Flight_2_duration_hours,Flight_2_End_Day,Flight 1 Start Date,Flight 1 End Date,Flight 2 Start Date,Flight 2 End Date,Waiting Time (hours),Waiting Days,Flight_2_Allocated_Members
0,No Return Double Tours (P3),Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,...,50,1.42,Tuesday,2025-05-05,2025-05-05,2025-05-06,2025-05-06,13.833333,0,Mo/MLE/102/CMB_Tu/CMB/173/BLR
1,No Return Double Tours (P3),Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,...,50,9.00,Tuesday,2025-05-05,2025-05-05,2025-05-06,2025-05-06,13.833333,0,Mo/MLE/102/CMB_Tu/CMB/306/SIN
2,No Return Double Tours (P3),Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,...,50,1.42,Wednesday,2025-05-05,2025-05-05,2025-05-07,2025-05-07,37.833333,1,Mo/MLE/102/CMB_We/CMB/173/BLR
3,No Return Double Tours (P3),Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,...,50,1.42,Thursday,2025-05-05,2025-05-05,2025-05-08,2025-05-08,61.833333,2,Mo/MLE/102/CMB_Th/CMB/173/BLR
4,No Return Double Tours (P3),Monday,UL 102,Male,MLE,Colombo,CMB,09:55:00,11:20:00,6,...,50,9.00,Thursday,2025-05-05,2025-05-05,2025-05-08,2025-05-08,61.833333,2,Mo/MLE/102/CMB_Th/CMB/306/SIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,No Return Double Tours (P3),Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,...,50,1.42,Wednesday,2025-05-04,2025-05-05,2025-05-07,2025-05-07,44.750000,1,Su/LHR/506/CMB_We/CMB/173/BLR
106,No Return Double Tours (P3),Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,...,50,1.42,Thursday,2025-05-04,2025-05-05,2025-05-08,2025-05-08,68.750000,2,Su/LHR/506/CMB_Th/CMB/173/BLR
107,No Return Double Tours (P3),Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,...,50,9.00,Thursday,2025-05-04,2025-05-05,2025-05-08,2025-05-08,68.750000,2,Su/LHR/506/CMB_Th/CMB/306/SIN
108,No Return Double Tours (P3),Sunday,UL 506,London,LHR,Colombo,CMB,18:05:00,04:25:00,6,...,50,9.00,Friday,2025-05-04,2025-05-05,2025-05-09,2025-05-09,92.750000,3,Su/LHR/506/CMB_Fr/CMB/306/SIN


In [ ]:
filtered_flights_1[['Tour Type', 'Flight_1_duration_hours', 'Flight_1_Allocated_Members', 'Flight_2_duration_hours', 'Waiting Days','Flight_2_Allocated_Members']]

,Tour Type,Flight_1_duration_hours,Flight_1_Allocated_Members,Flight_2_duration_hours,Waiting Days,Flight_2_Allocated_Members
0,No Return Double Tours (P3),1.42,Mo/MLE/102/CMB,1.42,0,Mo/MLE/102/CMB_Tu/CMB/173/BLR
1,No Return Double Tours (P3),1.42,Mo/MLE/102/CMB,9.00,0,Mo/MLE/102/CMB_Tu/CMB/306/SIN
2,No Return Double Tours (P3),1.42,Mo/MLE/102/CMB,1.42,1,Mo/MLE/102/CMB_We/CMB/173/BLR
3,No Return Double Tours (P3),1.42,Mo/MLE/102/CMB,1.42,2,Mo/MLE/102/CMB_Th/CMB/173/BLR
4,No Return Double Tours (P3),1.42,Mo/MLE/102/CMB,9.00,2,Mo/MLE/102/CMB_Th/CMB/306/SIN
...,...,...,...,...,...,...
105,No Return Double Tours (P3),10.33,Su/LHR/506/CMB,1.42,1,Su/LHR/506/CMB_We/CMB/173/BLR
106,No Return Double Tours (P3),10.33,Su/LHR/506/CMB,1.42,2,Su/LHR/506/CMB_Th/CMB/173/BLR
107,No Return Double Tours (P3),10.33,Su/LHR/506/CMB,9.00,2,Su/LHR/506/CMB_Th/CMB/306/SIN
108,No Return Double Tours (P3),10.33,Su/LHR/506/CMB,9.00,3,Su/LHR/506/CMB_Fr/CMB/306/SIN


In [ ]:
first_second = pd.read_csv("filtered_flights_1.csv")
first_second_third = pd.read_csv("filtered_flights_2.csv")
salary = pd.read_excel("salary.xlsx")

In [ ]:
first_second_selected = first_second[['Tour Type','Flight_1_Min', 'Flight_1_Max','Flight_1_duration_hours','Flight_1_Allocated_Members', 'Waiting Days','Flight_2_Min', 'Flight_2_Max', 'Flight_2_duration_hours','Flight_2_Allocated_Members']]

In [ ]:
first_second_selected["Total_salary"] = (
    first_second_selected["Flight_1_duration_hours"] * salary['payment'][0] +
    first_second_selected["Flight_2_duration_hours"] * salary['payment'][0] +
    first_second_selected["Waiting Days"] * salary['payment'][1]
)


<ipython-input-566-cc698b5da9ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_second_selected["Total_salary"] = (


In [ ]:
print(first_second_selected.isnull().sum())

Tour Type                     0
Flight_1_Min                  0
Flight_1_Max                  0
Flight_1_duration_hours       0
Flight_1_Allocated_Members    0
Waiting Days                  0
Flight_2_Min                  0
Flight_2_Max                  0
Flight_2_duration_hours       0
Flight_2_Allocated_Members    0
Total_salary                  0
dtype: int64


In [ ]:
first_second_selected

,Tour Type,Flight_1_Min,Flight_1_Max,Flight_1_duration_hours,Flight_1_Allocated_Members,Waiting Days,Flight_2_Min,Flight_2_Max,Flight_2_duration_hours,Flight_2_Allocated_Members,Total_salary
0,No Return Double Tours (P3),6,50,1.42,Mo/MLE/102/CMB,0,3,50,1.42,Mo/MLE/102/CMB_Tu/CMB/173/BLR,710.0
1,No Return Double Tours (P3),6,50,1.42,Mo/MLE/102/CMB,0,3,50,9.00,Mo/MLE/102/CMB_Tu/CMB/306/SIN,2605.0
2,No Return Double Tours (P3),6,50,1.42,Mo/MLE/102/CMB,1,3,50,1.42,Mo/MLE/102/CMB_We/CMB/173/BLR,725.0
3,No Return Double Tours (P3),6,50,1.42,Mo/MLE/102/CMB,2,3,50,1.42,Mo/MLE/102/CMB_Th/CMB/173/BLR,740.0
4,No Return Double Tours (P3),6,50,1.42,Mo/MLE/102/CMB,2,3,50,9.00,Mo/MLE/102/CMB_Th/CMB/306/SIN,2635.0
...,...,...,...,...,...,...,...,...,...,...,...
105,No Return Double Tours (P3),6,50,10.33,Su/LHR/506/CMB,1,3,50,1.42,Su/LHR/506/CMB_We/CMB/173/BLR,2952.5
106,No Return Double Tours (P3),6,50,10.33,Su/LHR/506/CMB,2,3,50,1.42,Su/LHR/506/CMB_Th/CMB/173/BLR,2967.5
107,No Return Double Tours (P3),6,50,10.33,Su/LHR/506/CMB,2,3,50,9.00,Su/LHR/506/CMB_Th/CMB/306/SIN,4862.5
108,No Return Double Tours (P3),6,50,10.33,Su/LHR/506/CMB,3,3,50,9.00,Su/LHR/506/CMB_Fr/CMB/306/SIN,4877.5


In [ ]:
# flight 1 unique member names index
unique_indices = first_second_selected.loc[first_second_selected["Flight_1_Allocated_Members"].drop_duplicates().index].index #To use the variable_1

In [ ]:
unique_indices #Flight 1 crew mwmbwes index

Index([  0,   7,  11,  12,  17,  23,  28,  32,  38,  44,  48,  50,  55,  60,
        61,  67,  73,  77,  83,  89,  96, 103],
      dtype='int64')

In [ ]:
# Flight 1 boundaries
# get that unique flight 1 member name in the first_second_select df flight_1 allocated_members
variable_1=[]
for i in unique_indices:
    variable_1.append(first_second_selected["Flight_1_Allocated_Members"][i])

# get that unique flight 1 member name in the first_second_select df flight_1 allocated_memberstotal salary
Unique_members_of_Flight_1_salary=[]
for i in unique_indices:
    Unique_members_of_Flight_1_salary.append(first_second_selected["Total_salary"][i])

# the flight 1 allocated members lower boundary
lower_bound1 = []
upper_bound1 = []
for i in unique_indices:
    lower_bound1.append(first_second_selected['Flight_1_Min'][i])
    upper_bound1.append(first_second_selected['Flight_1_Max'][i])

flight1_details = pd.DataFrame({
    "Flight_1_Allocated_Members": variable_1,
    "Total_salary": Unique_members_of_Flight_1_salary,
    "Flight_1_Min": lower_bound1,
    "Flight_1_Max": upper_bound1
})

flight1_details

,Flight_1_Allocated_Members,Total_salary,Flight_1_Min,Flight_1_Max
0,Mo/MLE/102/CMB,710.0,6,50
1,Tu/CMB/173/BLR,730.0,3,50
2,Tu/CMB/306/SIN,3447.5,3,50
3,Tu/BLR/174/CMB,730.0,3,50
4,Tu/SIN/307/MLE,1597.5,6,50
5,Tu/MLE/102/CMB,710.0,6,50
6,We/CMB/173/BLR,820.0,3,50
7,We/BLR/174/CMB,805.0,3,50
8,We/MLE/102/CMB,785.0,6,50
9,Th/CMB/173/BLR,805.0,3,50


In [ ]:
# that unique flight 1 members assignment into flight 2
# Dictionary to store indices separately
one_to_2_indices_dict = {}

for j in variable_1:
    one_to_2_indices_dict[j] = first_second_selected[first_second_selected['Flight_2_Allocated_Members'].str.startswith(j, na=False)].index.tolist()

# Print results
for j, indices in one_to_2_indices_dict.items():
    print(f"Indices for 1st flight '{j}': {indices}")

Indices for 1st flight 'Mo/MLE/102/CMB': [0, 1, 2, 3, 4, 5, 6]
Indices for 1st flight 'Tu/CMB/173/BLR': [7, 8, 9, 10]
Indices for 1st flight 'Tu/CMB/306/SIN': [11]
Indices for 1st flight 'Tu/BLR/174/CMB': [12, 13, 14, 15, 16]
Indices for 1st flight 'Tu/SIN/307/MLE': [17, 18, 19, 20, 21, 22]
Indices for 1st flight 'Tu/MLE/102/CMB': [23, 24, 25, 26, 27]
Indices for 1st flight 'We/CMB/173/BLR': [28, 29, 30, 31]
Indices for 1st flight 'We/BLR/174/CMB': [32, 33, 34, 35, 36, 37]
Indices for 1st flight 'We/MLE/102/CMB': [38, 39, 40, 41, 42, 43]
Indices for 1st flight 'Th/CMB/173/BLR': [44, 45, 46, 47]
Indices for 1st flight 'Th/CMB/306/SIN': [48, 49]
Indices for 1st flight 'Th/BLR/174/CMB': [50, 51, 52, 53, 54]
Indices for 1st flight 'Th/MLE/102/CMB': [55, 56, 57, 58, 59]
Indices for 1st flight 'Fr/CMB/306/SIN': [60]
Indices for 1st flight 'Fr/SIN/307/MLE': [61, 62, 63, 64, 65, 66]
Indices for 1st flight 'Fr/MLE/102/CMB': [67, 68, 69, 70, 71, 72]
Indices for 1st flight 'Sa/CMB/173/BLR': [73, 

In [ ]:
one_to_2_indices_dict = pd.DataFrame([(key, indices) for key, indices in one_to_2_indices_dict.items()], columns=["Member", "Indices"])
one_to_2_indices_dict.to_csv("one_to_2_indices_dict.csv", index=False)

In [ ]:
one_to_2_indices_dict

,Member,Indices
0,Mo/MLE/102/CMB,"[0, 1, 2, 3, 4, 5, 6]"
1,Tu/CMB/173/BLR,"[7, 8, 9, 10]"
2,Tu/CMB/306/SIN,[11]
3,Tu/BLR/174/CMB,"[12, 13, 14, 15, 16]"
4,Tu/SIN/307/MLE,"[17, 18, 19, 20, 21, 22]"
5,Tu/MLE/102/CMB,"[23, 24, 25, 26, 27]"
6,We/CMB/173/BLR,"[28, 29, 30, 31]"
7,We/BLR/174/CMB,"[32, 33, 34, 35, 36, 37]"
8,We/MLE/102/CMB,"[38, 39, 40, 41, 42, 43]"
9,Th/CMB/173/BLR,"[44, 45, 46, 47]"


In [ ]:
# # the flight 2 allocated members lower boundary
# lower_bound2 = []
# upper_bound2 = []
# for i in range(len(first_second_selected)):
#     lower_bound2.append(first_second_selected['Flight_2_Min'][i])
#     upper_bound2.append(first_second_selected['Flight_2_Max'][i])

# flight 2 lowe and upper boundary
variables_2=[]
# lower_bound_2 = []
# upper_bound_2 = []
for i in range(len(one_to_2_indices_dict)):
  for j in one_to_2_indices_dict['Indices'][i]:
    variables_2.append(first_second_selected['Flight_2_Allocated_Members'][j])
    # lower_bound_2.append(lower_bound2[j])
    # upper_bound_2.append(upper_bound2[j])

In [ ]:
salary_flight_2=[]
for i in first_second_selected['Total_salary']:
  salary_flight_2.append(i)
print(salary_flight_2)

[710.0, 2605.0, 725.0, 740.0, 2635.0, 2650.0, 770.0, 730.0, 745.0, 760.0, 790.0, 3447.5, 730.0, 745.0, 2640.0, 2655.0, 775.0, 1597.5, 1522.5, 1537.5, 1552.5, 1567.5, 1582.5, 710.0, 725.0, 2620.0, 2635.0, 755.0, 820.0, 730.0, 745.0, 775.0, 805.0, 2700.0, 730.0, 2625.0, 2640.0, 760.0, 785.0, 2680.0, 710.0, 2605.0, 2620.0, 740.0, 805.0, 820.0, 730.0, 760.0, 3477.5, 3417.5, 790.0, 2685.0, 805.0, 2625.0, 745.0, 770.0, 2665.0, 785.0, 2605.0, 725.0, 3462.5, 1552.5, 1567.5, 1582.5, 1597.5, 1522.5, 1537.5, 755.0, 2650.0, 770.0, 785.0, 2680.0, 710.0, 775.0, 790.0, 805.0, 730.0, 760.0, 2655.0, 775.0, 790.0, 2685.0, 2700.0, 740.0, 2635.0, 755.0, 770.0, 2665.0, 2680.0, 2952.5, 4847.5, 2967.5, 2982.5, 4877.5, 4892.5, 3012.5, 725.0, 2620.0, 740.0, 755.0, 2650.0, 2665.0, 785.0, 2937.5, 4832.5, 2952.5, 2967.5, 4862.5, 4877.5, 2997.5]


In [ ]:
flight_2_members=[]
for i in first_second_selected['Flight_2_Allocated_Members']:
  flight_2_members.append(i)
print(flight_2_members)

['Mo/MLE/102/CMB_Tu/CMB/173/BLR', 'Mo/MLE/102/CMB_Tu/CMB/306/SIN', 'Mo/MLE/102/CMB_We/CMB/173/BLR', 'Mo/MLE/102/CMB_Th/CMB/173/BLR', 'Mo/MLE/102/CMB_Th/CMB/306/SIN', 'Mo/MLE/102/CMB_Fr/CMB/306/SIN', 'Mo/MLE/102/CMB_Sa/CMB/173/BLR', 'Tu/CMB/173/BLR_Tu/BLR/174/CMB', 'Tu/CMB/173/BLR_We/BLR/174/CMB', 'Tu/CMB/173/BLR_Th/BLR/174/CMB', 'Tu/CMB/173/BLR_Sa/BLR/174/CMB', 'Tu/CMB/306/SIN_Fr/SIN/307/MLE', 'Tu/BLR/174/CMB_We/CMB/173/BLR', 'Tu/BLR/174/CMB_Th/CMB/173/BLR', 'Tu/BLR/174/CMB_Th/CMB/306/SIN', 'Tu/BLR/174/CMB_Fr/CMB/306/SIN', 'Tu/BLR/174/CMB_Sa/CMB/173/BLR', 'Tu/SIN/307/MLE_Mo/MLE/102/CMB', 'Tu/SIN/307/MLE_We/MLE/102/CMB', 'Tu/SIN/307/MLE_Th/MLE/102/CMB', 'Tu/SIN/307/MLE_Fr/MLE/102/CMB', 'Tu/SIN/307/MLE_Sa/MLE/102/CMB', 'Tu/SIN/307/MLE_Su/MLE/102/CMB', 'Tu/MLE/102/CMB_We/CMB/173/BLR', 'Tu/MLE/102/CMB_Th/CMB/173/BLR', 'Tu/MLE/102/CMB_Th/CMB/306/SIN', 'Tu/MLE/102/CMB_Fr/CMB/306/SIN', 'Tu/MLE/102/CMB_Sa/CMB/173/BLR', 'We/CMB/173/BLR_Tu/BLR/174/CMB', 'We/CMB/173/BLR_We/BLR/174/CMB', 'We/CMB/1

In [ ]:
# Calculate count of indices for each member
one_to_2_indices_dict["Indices_Count"] = one_to_2_indices_dict["Indices"].apply(len)
one_to_2_indices_dict

,Member,Indices,Indices_Count
0,Mo/MLE/102/CMB,"[0, 1, 2, 3, 4, 5, 6]",7
1,Tu/CMB/173/BLR,"[7, 8, 9, 10]",4
2,Tu/CMB/306/SIN,[11],1
3,Tu/BLR/174/CMB,"[12, 13, 14, 15, 16]",5
4,Tu/SIN/307/MLE,"[17, 18, 19, 20, 21, 22]",6
5,Tu/MLE/102/CMB,"[23, 24, 25, 26, 27]",5
6,We/CMB/173/BLR,"[28, 29, 30, 31]",4
7,We/BLR/174/CMB,"[32, 33, 34, 35, 36, 37]",6
8,We/MLE/102/CMB,"[38, 39, 40, 41, 42, 43]",6
9,Th/CMB/173/BLR,"[44, 45, 46, 47]",4


In [ ]:
first_second_third_selected = first_second_third[['Tour Type','Flight_1_Min', 'Flight_1_Max','Flight_1_duration_hours', 'Waiting Days','Flight_2_Min', 'Flight_2_Max', 'Flight_2_duration_hours','Flight_2_Allocated_Members', 'Flight_3_Min', 'Flight_3_Max', 'Flight_3_duration_hours', 'Waiting_Days_2', 'Flight_3_Allocated_Members' ]]

In [ ]:
print(first_second_third_selected.isnull().sum())

Tour Type                      0
Flight_1_Min                   0
Flight_1_Max                   0
Flight_1_duration_hours        0
Waiting Days                   0
Flight_2_Min                   0
Flight_2_Max                   0
Flight_2_duration_hours        0
Flight_2_Allocated_Members     0
Flight_3_Min                  32
Flight_3_Max                  32
Flight_3_duration_hours       32
Waiting_Days_2                32
Flight_3_Allocated_Members     0
dtype: int64


In [ ]:
first_second_third_selected.fillna(0, inplace=True)
print(first_second_third_selected.isnull().sum())

Tour Type                     0
Flight_1_Min                  0
Flight_1_Max                  0
Flight_1_duration_hours       0
Waiting Days                  0
Flight_2_Min                  0
Flight_2_Max                  0
Flight_2_duration_hours       0
Flight_2_Allocated_Members    0
Flight_3_Min                  0
Flight_3_Max                  0
Flight_3_duration_hours       0
Waiting_Days_2                0
Flight_3_Allocated_Members    0
dtype: int64


<ipython-input-581-3569d42bff3e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_second_third_selected.fillna(0, inplace=True)


In [ ]:
# Salary Calculation
if (first_second_third_selected["Tour Type"] == "No Return Triple Tours (P4)").any():
    first_second_third_selected.loc[first_second_third_selected["Tour Type"] == "No Return Triple Tours (P4)", "Total_salary"] = (
        (first_second_third_selected["Flight_1_duration_hours"] + first_second_third_selected["Flight_2_duration_hours"] + first_second_third_selected["Flight_3_duration_hours"]) * salary['payment'][0] + (first_second_third_selected["Waiting Days"] + first_second_third_selected["Waiting_Days_2"]) * salary['payment'][1] + salary['payment'][2]
    )
if(first_second_third_selected["Tour Type"] == "No Return double Tours (P3)").any():
    first_second_third_selected.loc[first_second_third_selected["Tour Type"] == "No Return double Tours (P3)", "Total_salary"] = (
      ((first_second_third_selected["Flight_1_duration_hours"] + first_second_third_selected["Flight_2_duration_hours"])*salary['payment'][0]) + ((first_second_third_selected["Waiting Days"] ) * salary['payment'][1]) + (salary['payment'][2])
    )
if (first_second_third_selected["Tour Type"] == "Return Triple Tours (C3)").any():
    first_second_third_selected.loc[first_second_third_selected["Tour Type"] == "Return Triple Tours (C3)", "Total_salary"] = (
        (first_second_third_selected["Flight_1_duration_hours"] + first_second_third_selected["Flight_2_duration_hours"] + first_second_third_selected["Flight_3_duration_hours"]) * salary['payment'][0] + (first_second_third_selected["Waiting Days"] + first_second_third_selected["Waiting_Days_2"]) * salary['payment'][1]
    )

<ipython-input-582-992041d10e94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_second_third_selected.loc[first_second_third_selected["Tour Type"] == "No Return Triple Tours (P4)", "Total_salary"] = (


In [ ]:
unique_indices_2 = first_second_third_selected.loc[first_second_third_selected["Flight_2_Allocated_Members"].drop_duplicates().index].index

In [ ]:
Unique_members_of_Flight_2=[]
for i in unique_indices_2:
    Unique_members_of_Flight_2.append(first_second_third_selected["Flight_2_Allocated_Members"][i])

In [ ]:
# Dictionary to store indices separately
one_to_2_3_indices_dict = {}

for j in Unique_members_of_Flight_2:
    one_to_2_3_indices_dict[j] = first_second_third_selected[first_second_third_selected['Flight_3_Allocated_Members'].str.startswith(j, na=False)].index.tolist()

# Print results
for j, indices in one_to_2_3_indices_dict.items():
    print(f"Indices for 2st flight_first_second_third_selected '{j}': {indices}")

Indices for 2st flight_first_second_third_selected 'Mo/MLE/102/CMB_Tu/CMB/306/SIN': [0, 32]
Indices for 2st flight_first_second_third_selected 'Mo/MLE/102/CMB_Th/CMB/306/SIN': [1, 33]
Indices for 2st flight_first_second_third_selected 'Tu/CMB/306/SIN_Fr/SIN/307/MLE': [2, 34, 35, 36]
Indices for 2st flight_first_second_third_selected 'Tu/BLR/174/CMB_Th/CMB/306/SIN': [3, 37]
Indices for 2st flight_first_second_third_selected 'Tu/SIN/307/MLE_We/MLE/102/CMB': [4, 38, 39, 40, 41]
Indices for 2st flight_first_second_third_selected 'Tu/SIN/307/MLE_Th/MLE/102/CMB': [5, 42, 43]
Indices for 2st flight_first_second_third_selected 'Tu/SIN/307/MLE_Fr/MLE/102/CMB': [6, 44]
Indices for 2st flight_first_second_third_selected 'Tu/MLE/102/CMB_Th/CMB/306/SIN': [7, 45]
Indices for 2st flight_first_second_third_selected 'We/BLR/174/CMB_Th/CMB/306/SIN': [8, 46, 47]
Indices for 2st flight_first_second_third_selected 'We/BLR/174/CMB_Fr/CMB/306/SIN': [9, 48]
Indices for 2st flight_first_second_third_selected '

In [ ]:
one_to_2_3_indices_dict = pd.DataFrame([(key, indices) for key, indices in one_to_2_3_indices_dict.items()], columns=["Member", "Indices"])
one_to_2_3_indices_dict.to_csv("one_to_2_3_indices_dict.csv", index=False)

In [ ]:
one_to_2_3_indices_dict

,Member,Indices
0,Mo/MLE/102/CMB_Tu/CMB/306/SIN,"[0, 32]"
1,Mo/MLE/102/CMB_Th/CMB/306/SIN,"[1, 33]"
2,Tu/CMB/306/SIN_Fr/SIN/307/MLE,"[2, 34, 35, 36]"
3,Tu/BLR/174/CMB_Th/CMB/306/SIN,"[3, 37]"
4,Tu/SIN/307/MLE_We/MLE/102/CMB,"[4, 38, 39, 40, 41]"
5,Tu/SIN/307/MLE_Th/MLE/102/CMB,"[5, 42, 43]"
6,Tu/SIN/307/MLE_Fr/MLE/102/CMB,"[6, 44]"
7,Tu/MLE/102/CMB_Th/CMB/306/SIN,"[7, 45]"
8,We/BLR/174/CMB_Th/CMB/306/SIN,"[8, 46, 47]"
9,We/BLR/174/CMB_Fr/CMB/306/SIN,"[9, 48]"


In [ ]:
variables_3= first_second_third_selected['Flight_3_Allocated_Members']

In [ ]:
salary_flight_3=[]
for i in first_second_third_selected['Total_salary']:
  salary_flight_3.append(i)
print(salary_flight_3)

[2630.0, 2660.0, 3472.5, 2665.0, 1547.5, 1562.5, 1577.5, 2645.0, 2650.0, 2665.0, 2630.0, 2645.0, 3502.5, 3442.5, 2650.0, 2630.0, 3487.5, 1577.5, 1592.5, 1607.5, 1547.5, 1562.5, 2680.0, 2710.0, 2660.0, 2690.0, 4872.5, 4902.5, 2645.0, 2675.0, 4857.5, 4887.5, 3802.5, 3802.5, 3832.5, 3802.5, 3817.5, 3832.5, 1902.5, 3772.5, 3787.5, 1932.5, 3787.5, 1932.5, 1932.5, 3787.5, 3877.5, 3817.5, 3877.5, 3832.5, 3772.5, 3832.5, 3832.5, 3802.5, 3817.5, 3832.5, 3772.5, 3787.5, 3862.5, 3817.5, 3817.5, 3832.5, 1932.5, 3802.5, 1947.5, 1962.5, 3832.5, 1947.5, 1962.5, 3832.5, 1962.5, 3832.5, 1932.5, 3802.5, 1947.5, 1962.5, 3832.5, 1932.5, 3802.5, 1947.5, 1962.5, 3832.5, 3877.5, 3877.5, 3832.5, 3832.5, 6070.0, 6070.0, 3817.5, 3817.5, 6055.0, 6055.0]


In [ ]:
flight_3_members=[]
for i in first_second_third_selected['Flight_3_Allocated_Members']:
  flight_3_members.append(i)
print(flight_3_members)

['Mo/MLE/102/CMB_Tu/CMB/306/SIN_NaN', 'Mo/MLE/102/CMB_Th/CMB/306/SIN_NaN', 'Tu/CMB/306/SIN_Fr/SIN/307/MLE_NaN', 'Tu/BLR/174/CMB_Th/CMB/306/SIN_NaN', 'Tu/SIN/307/MLE_We/MLE/102/CMB_NaN', 'Tu/SIN/307/MLE_Th/MLE/102/CMB_NaN', 'Tu/SIN/307/MLE_Fr/MLE/102/CMB_NaN', 'Tu/MLE/102/CMB_Th/CMB/306/SIN_NaN', 'We/BLR/174/CMB_Th/CMB/306/SIN_NaN', 'We/BLR/174/CMB_Fr/CMB/306/SIN_NaN', 'We/MLE/102/CMB_Th/CMB/306/SIN_NaN', 'We/MLE/102/CMB_Fr/CMB/306/SIN_NaN', 'Th/CMB/306/SIN_Tu/SIN/307/MLE_NaN', 'Th/CMB/306/SIN_Fr/SIN/307/MLE_NaN', 'Th/BLR/174/CMB_Fr/CMB/306/SIN_NaN', 'Th/MLE/102/CMB_Fr/CMB/306/SIN_NaN', 'Fr/CMB/306/SIN_Tu/SIN/307/MLE_NaN', 'Fr/SIN/307/MLE_Mo/MLE/102/CMB_NaN', 'Fr/SIN/307/MLE_Tu/MLE/102/CMB_NaN', 'Fr/SIN/307/MLE_We/MLE/102/CMB_NaN', 'Fr/SIN/307/MLE_Sa/MLE/102/CMB_NaN', 'Fr/SIN/307/MLE_Su/MLE/102/CMB_NaN', 'Sa/BLR/174/CMB_Tu/CMB/306/SIN_NaN', 'Sa/BLR/174/CMB_Th/CMB/306/SIN_NaN', 'Sa/MLE/102/CMB_Tu/CMB/306/SIN_NaN', 'Sa/MLE/102/CMB_Th/CMB/306/SIN_NaN', 'Sa/LHR/506/CMB_Tu/CMB/306/SIN_NaN', 

In [ ]:
# Calculate count of indices for each member
one_to_2_3_indices_dict["Indices_Count2"] = one_to_2_3_indices_dict["Indices"].apply(len)
one_to_2_3_indices_dict

,Member,Indices,Indices_Count2
0,Mo/MLE/102/CMB_Tu/CMB/306/SIN,"[0, 32]",2
1,Mo/MLE/102/CMB_Th/CMB/306/SIN,"[1, 33]",2
2,Tu/CMB/306/SIN_Fr/SIN/307/MLE,"[2, 34, 35, 36]",4
3,Tu/BLR/174/CMB_Th/CMB/306/SIN,"[3, 37]",2
4,Tu/SIN/307/MLE_We/MLE/102/CMB,"[4, 38, 39, 40, 41]",5
5,Tu/SIN/307/MLE_Th/MLE/102/CMB,"[5, 42, 43]",3
6,Tu/SIN/307/MLE_Fr/MLE/102/CMB,"[6, 44]",2
7,Tu/MLE/102/CMB_Th/CMB/306/SIN,"[7, 45]",2
8,We/BLR/174/CMB_Th/CMB/306/SIN,"[8, 46, 47]",3
9,We/BLR/174/CMB_Fr/CMB/306/SIN,"[9, 48]",2


In [ ]:
# Convert to DataFrame with explicit column name
df = pd.DataFrame(flight_2_members, columns=["Flight_2_Allocated_Members"])

# Function to extract substring after "_"
def extract_after_underscore(value):
    if isinstance(value, str) and "_" in value:
        return value.split("_", 1)[1]  # Get part after "_"
    return None  # Return None if "_" is not found

# Extract the second part
df['extracted_part'] = df["Flight_2_Allocated_Members"].apply(extract_after_underscore)

# Drop duplicate extracted parts but keep first occurrence
df_unique = df[['extracted_part']].drop_duplicates().dropna().reset_index(drop=True)

# Get the indices where each extracted part appears (ensuring exact suffix matching)
df_unique['full_part_index'] = df_unique['extracted_part'].apply(
    lambda x: df.index[df['Flight_2_Allocated_Members'].apply(
        lambda y: y.endswith(x) if isinstance(y, str) else False
    )].tolist()
)

# Get the count of occurrences for each extracted part
df_unique['count'] = df_unique['full_part_index'].apply(len)

# Display the result
df_unique

,extracted_part,full_part_index,count
0,Tu/CMB/173/BLR,"[0, 32, 38, 50, 55, 67, 77, 83, 89, 96, 103]",11
1,Tu/CMB/306/SIN,"[1, 33, 39, 51, 56, 68, 78, 84, 90, 97, 104]",11
2,We/CMB/173/BLR,"[2, 12, 23, 52, 57, 69, 79, 85, 91, 98, 105]",11
3,Th/CMB/173/BLR,"[3, 13, 24, 34, 40, 70, 80, 86, 92, 99, 106]",11
4,Th/CMB/306/SIN,"[4, 14, 25, 35, 41, 71, 81, 87, 93, 100, 107]",11
5,Fr/CMB/306/SIN,"[5, 15, 26, 36, 42, 53, 58, 82, 88, 94, 101, 108]",12
6,Sa/CMB/173/BLR,"[6, 16, 27, 37, 43, 54, 59, 72, 95, 102, 109]",11
7,Tu/BLR/174/CMB,"[7, 28, 44, 73]",4
8,We/BLR/174/CMB,"[8, 29, 45, 74]",4
9,Th/BLR/174/CMB,"[9, 30, 46, 75]",4


In [ ]:
flight_2_members = first_second_selected["Flight_2_Allocated_Members"].tolist()
# Find common variable names directly
common_variable_names = set(flight_2_members) & set(one_to_2_3_indices_dict['Member'])
common_variable_names = list(common_variable_names)
# Create a dictionary for variable2_2 with common variable names as keys and counts as lower bounds
variables2_2 = {name: (0, one_to_2_3_indices_dict['Indices_Count2'][i])
               for i, name in enumerate(one_to_2_3_indices_dict['Member'])
               if name in common_variable_names}

# Get indices of common variable names in first_second_selected["Flight_2_Allocated_Members"]
common_indices = [i for i, member in enumerate(flight_2_members) if member in common_variable_names]

variables2_2
from pulp import LpVariable

# Create LpVariables for variables2_2 with the correct lower bound values
variables2_2 = {name: LpVariable(name, lowBound=count, cat='Integer')
                for name, count in zip(one_to_2_3_indices_dict['Member'], one_to_2_3_indices_dict['Indices_Count2'])
                if name in common_variable_names}

# Drop the common elements from flight_2_members
remaining_flight_2_members = [member for member in flight_2_members if member not in common_variable_names]
remaining_flight_2_members = list(remaining_flight_2_members)
# Define variables2_1 using the remaining flight_2_members as names
variables2_1 = {
    member: LpVariable(
        member,
        lowBound=0,
        cat='Integer'
    )
    for member in remaining_flight_2_members
}
remaining_indices = [i for i, member in enumerate(flight_2_members) if member in remaining_flight_2_members]

In [ ]:
one_to_2_3_indices_dict


,Member,Indices,Indices_Count2
0,Mo/MLE/102/CMB_Tu/CMB/306/SIN,"[0, 32]",2
1,Mo/MLE/102/CMB_Th/CMB/306/SIN,"[1, 33]",2
2,Tu/CMB/306/SIN_Fr/SIN/307/MLE,"[2, 34, 35, 36]",4
3,Tu/BLR/174/CMB_Th/CMB/306/SIN,"[3, 37]",2
4,Tu/SIN/307/MLE_We/MLE/102/CMB,"[4, 38, 39, 40, 41]",5
5,Tu/SIN/307/MLE_Th/MLE/102/CMB,"[5, 42, 43]",3
6,Tu/SIN/307/MLE_Fr/MLE/102/CMB,"[6, 44]",2
7,Tu/MLE/102/CMB_Th/CMB/306/SIN,"[7, 45]",2
8,We/BLR/174/CMB_Th/CMB/306/SIN,"[8, 46, 47]",3
9,We/BLR/174/CMB_Fr/CMB/306/SIN,"[9, 48]",2


In [ ]:
len(remaining_flight_2_members)

78

In [ ]:
len(common_variable_names)

32

In [ ]:
# Merge on extracted_part (df_unique) and Members (flight1_details)
merged_df = df_unique.merge(flight1_details, left_on='extracted_part', right_on='Flight_1_Allocated_Members', how='left')

# Drop the 'Members' column and any other columns from flight1_details you don't want
merged_df = merged_df.drop(columns=['Flight_1_Allocated_Members', 'Total_salary'])

# Display the result
merged_df

,extracted_part,full_part_index,count,Flight_1_Min,Flight_1_Max
0,Tu/CMB/173/BLR,"[0, 32, 38, 50, 55, 67, 77, 83, 89, 96, 103]",11,3,50
1,Tu/CMB/306/SIN,"[1, 33, 39, 51, 56, 68, 78, 84, 90, 97, 104]",11,3,50
2,We/CMB/173/BLR,"[2, 12, 23, 52, 57, 69, 79, 85, 91, 98, 105]",11,3,50
3,Th/CMB/173/BLR,"[3, 13, 24, 34, 40, 70, 80, 86, 92, 99, 106]",11,3,50
4,Th/CMB/306/SIN,"[4, 14, 25, 35, 41, 71, 81, 87, 93, 100, 107]",11,3,50
5,Fr/CMB/306/SIN,"[5, 15, 26, 36, 42, 53, 58, 82, 88, 94, 101, 108]",12,3,50
6,Sa/CMB/173/BLR,"[6, 16, 27, 37, 43, 54, 59, 72, 95, 102, 109]",11,3,50
7,Tu/BLR/174/CMB,"[7, 28, 44, 73]",4,3,50
8,We/BLR/174/CMB,"[8, 29, 45, 74]",4,3,50
9,Th/BLR/174/CMB,"[9, 30, 46, 75]",4,3,50


In [ ]:
salary_flight_2_2=[]
for i in common_indices:
  salary_flight_2_2.append(first_second_selected['Total_salary'][i])
salary_flight_2_1=[]
for j in remaining_indices:
  salary_flight_2_1.append(first_second_selected['Total_salary'][j])

In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
# Define the problem
problem = LpProblem("Crew_Optimization", LpMinimize)

In [ ]:
# Preprocess salaries to replace None with 0
salary_flight_2_2 = [s if s is not None else 0 for s in salary_flight_2_2]
salary_flight_2_1 = [s if s is not None else 0 for s in salary_flight_2_1]

# Define weights for objective function
alpha = 1     # Cost minimization weight
beta = -0.1   # Crew utilization encouragement (adjustable)

# Objective function: Minimize cost while maximizing crew allocation
problem += (
    alpha * lpSum([
        salary_flight_2_2[i] * variables2_2.get(common_variable_names[i], 0)
        for i in range(len(common_variable_names))
    ]) +
    alpha * lpSum([
        salary_flight_2_1[i] * variables2_1.get(remaining_flight_2_members[i], 0)
        for i in range(len(remaining_flight_2_members))
    ]) +
    beta * (lpSum(variables2_2.values()) + lpSum(variables2_1.values()))
), "Total Cost and Maximum Utilization"


In [ ]:
problem

Crew_Optimization:
MINIMIZE
2634.9*Fr_CMB_306_SIN_Tu_SIN_307_MLE + 709.9*Fr_MLE_102_CMB_Sa_CMB_173_BLR + 784.9*Fr_MLE_102_CMB_Th_CMB_173_BLR + 2679.9*Fr_MLE_102_CMB_Th_CMB_306_SIN + 754.9*Fr_MLE_102_CMB_Tu_CMB_173_BLR + 2649.9*Fr_MLE_102_CMB_Tu_CMB_306_SIN + 769.9*Fr_MLE_102_CMB_We_CMB_173_BLR + 2624.9*Fr_SIN_307_MLE_Mo_MLE_102_CMB + 2619.9*Fr_SIN_307_MLE_Sa_MLE_102_CMB + 2619.9*Fr_SIN_307_MLE_Su_MLE_102_CMB + 1597.4*Fr_SIN_307_MLE_Th_MLE_102_CMB + 2639.9*Fr_SIN_307_MLE_Tu_MLE_102_CMB + 3447.4*Fr_SIN_307_MLE_We_MLE_102_CMB + 2649.9*Mo_MLE_102_CMB_Fr_CMB_306_SIN + 769.9*Mo_MLE_102_CMB_Sa_CMB_173_BLR + 739.9*Mo_MLE_102_CMB_Th_CMB_173_BLR + 2604.9*Mo_MLE_102_CMB_Th_CMB_306_SIN + 709.9*Mo_MLE_102_CMB_Tu_CMB_173_BLR + 2619.9*Mo_MLE_102_CMB_Tu_CMB_306_SIN + 724.9*Mo_MLE_102_CMB_We_CMB_173_BLR + 2699.9*Sa_BLR_174_CMB_Fr_CMB_306_SIN + 789.9*Sa_BLR_174_CMB_Th_CMB_173_BLR + 3462.4*Sa_BLR_174_CMB_Th_CMB_306_SIN + 759.9*Sa_BLR_174_CMB_Tu_CMB_173_BLR + 1537.4*Sa_BLR_174_CMB_Tu_CMB_306_SIN + 774.9*S

In [ ]:
one_to_2_3_indices_dict

,Member,Indices,Indices_Count2
0,Mo/MLE/102/CMB_Tu/CMB/306/SIN,"[0, 32]",2
1,Mo/MLE/102/CMB_Th/CMB/306/SIN,"[1, 33]",2
2,Tu/CMB/306/SIN_Fr/SIN/307/MLE,"[2, 34, 35, 36]",4
3,Tu/BLR/174/CMB_Th/CMB/306/SIN,"[3, 37]",2
4,Tu/SIN/307/MLE_We/MLE/102/CMB,"[4, 38, 39, 40, 41]",5
5,Tu/SIN/307/MLE_Th/MLE/102/CMB,"[5, 42, 43]",3
6,Tu/SIN/307/MLE_Fr/MLE/102/CMB,"[6, 44]",2
7,Tu/MLE/102/CMB_Th/CMB/306/SIN,"[7, 45]",2
8,We/BLR/174/CMB_Th/CMB/306/SIN,"[8, 46, 47]",3
9,We/BLR/174/CMB_Fr/CMB/306/SIN,"[9, 48]",2


In [ ]:
common_members = flight1_details['Flight_1_Allocated_Members'].isin(merged_df['extracted_part'])
common_index = flight1_details.index[common_members]
print(common_index)

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20], dtype='int64')


In [ ]:
all_indices_flight1 = flight1_details.index.tolist()
remaining_index = flight1_details.index[~flight1_details['Flight_1_Allocated_Members'].isin(merged_df['extracted_part'])]
print(remaining_index)

Index([19, 21], dtype='int64')


In [ ]:
min_values = []

# Get all relevant indices (0 to 25) from merged_df
all_indices = set(merged_df.index)

# Iterate over all indices (ensuring we include both common and remaining ones)
for i in all_indices:
    if i in common_indices:
        # If index is in common_indices, consider all three values
        min_values.append(max(
            one_to_2_indices_dict['Indices_Count'].get(i, 0),
            one_to_2_3_indices_dict['Indices_Count2'].get(i, 0),
            merged_df['count'].get(i, 0)  # Default to 0 if index not found
        ))
    else:
        # If index is NOT in common_indices, only consider the first two
        min_values.append(max(
            one_to_2_indices_dict['Indices_Count'].get(i, 0),
            one_to_2_3_indices_dict['Indices_Count2'].get(i, 0)
        ))

print(min_values)

# Create a mapping from Members to count in merged_df
member_to_count = dict(zip(merged_df['extracted_part'], merged_df['count']))

# Iterate over all expected indices (0 to 21)
for i in range(unique_count):
    member = one_to_2_indices_dict['Member'].get(i)  # Get corresponding Member

    if i in common_indices:
        min_values.append(max(
            one_to_2_indices_dict['Indices_Count'].get(i, 0),
            one_to_2_3_indices_dict['Indices_Count2'].get(i, 0),
            member_to_count.get(member, 0)# Map using Members instead of index
        ))
    else:
        min_values.append(max(
            one_to_2_indices_dict['Indices_Count'].get(i, 0),
            one_to_2_3_indices_dict['Indices_Count2'].get(i, 0)
        ))


[np.int64(7), np.int64(11), np.int64(4), np.int64(5), np.int64(11), np.int64(5), np.int64(4), np.int64(6), np.int64(6), np.int64(4), np.int64(3), np.int64(5), np.int64(5), np.int64(6), np.int64(6), np.int64(6), np.int64(4), np.int64(6), np.int64(6), np.int64(7)]


In [ ]:
variables2_2

{'Mo/MLE/102/CMB_Tu/CMB/306/SIN': Mo_MLE_102_CMB_Tu_CMB_306_SIN,
 'Mo/MLE/102/CMB_Th/CMB/306/SIN': Mo_MLE_102_CMB_Th_CMB_306_SIN,
 'Tu/CMB/306/SIN_Fr/SIN/307/MLE': Tu_CMB_306_SIN_Fr_SIN_307_MLE,
 'Tu/BLR/174/CMB_Th/CMB/306/SIN': Tu_BLR_174_CMB_Th_CMB_306_SIN,
 'Tu/SIN/307/MLE_We/MLE/102/CMB': Tu_SIN_307_MLE_We_MLE_102_CMB,
 'Tu/SIN/307/MLE_Th/MLE/102/CMB': Tu_SIN_307_MLE_Th_MLE_102_CMB,
 'Tu/SIN/307/MLE_Fr/MLE/102/CMB': Tu_SIN_307_MLE_Fr_MLE_102_CMB,
 'Tu/MLE/102/CMB_Th/CMB/306/SIN': Tu_MLE_102_CMB_Th_CMB_306_SIN,
 'We/BLR/174/CMB_Th/CMB/306/SIN': We_BLR_174_CMB_Th_CMB_306_SIN,
 'We/BLR/174/CMB_Fr/CMB/306/SIN': We_BLR_174_CMB_Fr_CMB_306_SIN,
 'We/MLE/102/CMB_Th/CMB/306/SIN': We_MLE_102_CMB_Th_CMB_306_SIN,
 'We/MLE/102/CMB_Fr/CMB/306/SIN': We_MLE_102_CMB_Fr_CMB_306_SIN,
 'Th/CMB/306/SIN_Tu/SIN/307/MLE': Th_CMB_306_SIN_Tu_SIN_307_MLE,
 'Th/CMB/306/SIN_Fr/SIN/307/MLE': Th_CMB_306_SIN_Fr_SIN_307_MLE,
 'Th/BLR/174/CMB_Fr/CMB/306/SIN': Th_BLR_174_CMB_Fr_CMB_306_SIN,
 'Th/MLE/102/CMB_Fr/CMB/3

In [ ]:
# Prepare max values
max_values1 = [
    max(flight1_details['Flight_1_Max'][i],
        one_to_2_indices_dict['Indices_Count'][i],
        one_to_2_3_indices_dict['Indices_Count2'][i])
    for i in range(len(one_to_2_indices_dict))
]

# Prepare min values
min_values = flight1_details['Flight_1_Min']

# Final max values, adjusted if necessary
max_values = []
for i in range(len(max_values1)):
    if max_values1[i] < min_values[i]:
        max_values.append(min_values[i])
    else:
        max_values.append(max_values1[i])
# _______END>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# Take user inputs within min/max bounds
user_inputs1 = {}

for i, key in enumerate(flight1_details['Flight_1_Allocated_Members']):
    m_min = min_values[i]
    m_max = max_values[i]
    while True:
        try:
            value = int(input(f"Enter value for {key} ({m_min}, {m_max}): "))
            if m_min <= value <= m_max:
                user_inputs1[key] = value
                break
            else:
                print(f"Value must be between {m_min} and {m_max}. Try again.")
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

print("Final user inputs:", user_inputs1)

Enter value for Mo/MLE/102/CMB (6, 50): 35
Enter value for Tu/CMB/173/BLR (3, 50): 30
Enter value for Tu/CMB/306/SIN (3, 50): 28
Enter value for Tu/BLR/174/CMB (3, 50): 36
Enter value for Tu/SIN/307/MLE (6, 50): 23
Enter value for Tu/MLE/102/CMB (6, 50): 25
Enter value for We/CMB/173/BLR (3, 50): 24
Enter value for We/BLR/174/CMB (3, 50): 26
Enter value for We/MLE/102/CMB (6, 50): 27
Enter value for Th/CMB/173/BLR (3, 50): 23
Enter value for Th/CMB/306/SIN (3, 50): 38
Enter value for Th/BLR/174/CMB (3, 50): 23
Enter value for Th/MLE/102/CMB (6, 50): 39
Enter value for Fr/CMB/306/SIN (3, 50): 24
Enter value for Fr/SIN/307/MLE (6, 50): 36
Enter value for Fr/MLE/102/CMB (6, 50): 34
Enter value for Sa/CMB/173/BLR (3, 50): 23
Enter value for Sa/BLR/174/CMB (3, 50): 24
Enter value for Sa/MLE/102/CMB (6, 50): 36
Enter value for Sa/LHR/506/CMB (6, 50): 32
Enter value for Su/MLE/102/CMB (6, 50): 38
Enter value for Su/LHR/506/CMB (6, 50): 39
Final user inputs: {'Mo/MLE/102/CMB': 35, 'Tu/CMB/173/

In [ ]:
indices_lst = []

for i in range(len(one_to_2_indices_dict)):
    row_indices = one_to_2_indices_dict["Indices"][i]
    indices_lst.append(row_indices)

print(indices_lst)


# problem += first_second_selected['Flight_2_Allocated_Members'][index]==var_value1[0]

[[0, 1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14, 15, 16], [17, 18, 19, 20, 21, 22], [23, 24, 25, 26, 27], [28, 29, 30, 31], [32, 33, 34, 35, 36, 37], [38, 39, 40, 41, 42, 43], [44, 45, 46, 47], [48, 49], [50, 51, 52, 53, 54], [55, 56, 57, 58, 59], [60], [61, 62, 63, 64, 65, 66], [67, 68, 69, 70, 71, 72], [73, 74, 75, 76], [77, 78, 79, 80, 81, 82], [83, 84, 85, 86, 87, 88], [89, 90, 91, 92, 93, 94, 95], [96, 97, 98, 99, 100, 101, 102], [103, 104, 105, 106, 107, 108, 109]]


In [ ]:
for i, indices in enumerate(indices_lst):
    selected_flight_2_members = first_second_selected.loc[indices, 'Flight_2_Allocated_Members']

    # Debugging print
    print(f"Index Group {i}: {indices}")
    print(f"Selected Flight 2 Members: {selected_flight_2_members.tolist()}")

    # Ensure all selected flights exist in either variables2 or variables4
    valid_variables = []
    missing_keys = []

    for flight in selected_flight_2_members:
        if flight in variables2_2:
            valid_variables.append(variables2_2[flight])
        elif flight in variables2_1:
            valid_variables.append(variables2_1[flight])
        else:
            missing_keys.append(flight)

    # Ensure Flight_1_Allocated_Members exist in user_inputs
    flight_1_key = first_second_selected['Flight_1_Allocated_Members'].iloc[i]
    if flight_1_key not in user_inputs1:
        print(f"Warning: Key {flight_1_key} missing from user_inputs")
        continue  # Skip this constraint if user_inputs key is missing

    # Print missing variables for debugging
    if missing_keys:
        print(f"Warning: These flights are missing from both variables2: {missing_keys}")

        # Only create constraint if valid variables exist
    if valid_variables:
        # Add the lower bound constraint: the sum of valid variables must be >= 1
        problem += lpSum(valid_variables) >= 1

        # # Add the upper bound constraint: the sum of valid variables must be <= user_inputs1[flight_1_key]
        problem += lpSum(valid_variables) >= user_inputs1[flight_1_key]


Index Group 0: [0, 1, 2, 3, 4, 5, 6]
Selected Flight 2 Members: ['Mo/MLE/102/CMB_Tu/CMB/173/BLR', 'Mo/MLE/102/CMB_Tu/CMB/306/SIN', 'Mo/MLE/102/CMB_We/CMB/173/BLR', 'Mo/MLE/102/CMB_Th/CMB/173/BLR', 'Mo/MLE/102/CMB_Th/CMB/306/SIN', 'Mo/MLE/102/CMB_Fr/CMB/306/SIN', 'Mo/MLE/102/CMB_Sa/CMB/173/BLR']
Index Group 1: [7, 8, 9, 10]
Selected Flight 2 Members: ['Tu/CMB/173/BLR_Tu/BLR/174/CMB', 'Tu/CMB/173/BLR_We/BLR/174/CMB', 'Tu/CMB/173/BLR_Th/BLR/174/CMB', 'Tu/CMB/173/BLR_Sa/BLR/174/CMB']
Index Group 2: [11]
Selected Flight 2 Members: ['Tu/CMB/306/SIN_Fr/SIN/307/MLE']
Index Group 3: [12, 13, 14, 15, 16]
Selected Flight 2 Members: ['Tu/BLR/174/CMB_We/CMB/173/BLR', 'Tu/BLR/174/CMB_Th/CMB/173/BLR', 'Tu/BLR/174/CMB_Th/CMB/306/SIN', 'Tu/BLR/174/CMB_Fr/CMB/306/SIN', 'Tu/BLR/174/CMB_Sa/CMB/173/BLR']
Index Group 4: [17, 18, 19, 20, 21, 22]
Selected Flight 2 Members: ['Tu/SIN/307/MLE_Mo/MLE/102/CMB', 'Tu/SIN/307/MLE_We/MLE/102/CMB', 'Tu/SIN/307/MLE_Th/MLE/102/CMB', 'Tu/SIN/307/MLE_Fr/MLE/102/CMB', 'Tu/S

In [ ]:
df_unique

,extracted_part,full_part_index,count
0,Tu/CMB/173/BLR,"[0, 32, 38, 50, 55, 67, 77, 83, 89, 96, 103]",11
1,Tu/CMB/306/SIN,"[1, 33, 39, 51, 56, 68, 78, 84, 90, 97, 104]",11
2,We/CMB/173/BLR,"[2, 12, 23, 52, 57, 69, 79, 85, 91, 98, 105]",11
3,Th/CMB/173/BLR,"[3, 13, 24, 34, 40, 70, 80, 86, 92, 99, 106]",11
4,Th/CMB/306/SIN,"[4, 14, 25, 35, 41, 71, 81, 87, 93, 100, 107]",11
5,Fr/CMB/306/SIN,"[5, 15, 26, 36, 42, 53, 58, 82, 88, 94, 101, 108]",12
6,Sa/CMB/173/BLR,"[6, 16, 27, 37, 43, 54, 59, 72, 95, 102, 109]",11
7,Tu/BLR/174/CMB,"[7, 28, 44, 73]",4
8,We/BLR/174/CMB,"[8, 29, 45, 74]",4
9,Th/BLR/174/CMB,"[9, 30, 46, 75]",4


In [ ]:
merged_df

,extracted_part,full_part_index,count,Flight_1_Min,Flight_1_Max
0,Tu/CMB/173/BLR,"[0, 32, 38, 50, 55, 67, 77, 83, 89, 96, 103]",11,3,50
1,Tu/CMB/306/SIN,"[1, 33, 39, 51, 56, 68, 78, 84, 90, 97, 104]",11,3,50
2,We/CMB/173/BLR,"[2, 12, 23, 52, 57, 69, 79, 85, 91, 98, 105]",11,3,50
3,Th/CMB/173/BLR,"[3, 13, 24, 34, 40, 70, 80, 86, 92, 99, 106]",11,3,50
4,Th/CMB/306/SIN,"[4, 14, 25, 35, 41, 71, 81, 87, 93, 100, 107]",11,3,50
5,Fr/CMB/306/SIN,"[5, 15, 26, 36, 42, 53, 58, 82, 88, 94, 101, 108]",12,3,50
6,Sa/CMB/173/BLR,"[6, 16, 27, 37, 43, 54, 59, 72, 95, 102, 109]",11,3,50
7,Tu/BLR/174/CMB,"[7, 28, 44, 73]",4,3,50
8,We/BLR/174/CMB,"[8, 29, 45, 74]",4,3,50
9,Th/BLR/174/CMB,"[9, 30, 46, 75]",4,3,50


In [ ]:
from pulp import lpSum

for i, row in df_unique.iterrows():
    indices = row['full_part_index']
    extracted_part = row['extracted_part']

    if not indices:
        continue  # Skip if no matching indices

    # Extract selected flight members based on the indices
    selected_flight_2_members = first_second_selected.loc[indices, "Flight_2_Allocated_Members"]
    # Debugging print
    print(f"Index Group {i}: {indices}")
    print(f"Selected Flight 2 Members: {selected_flight_2_members.tolist()}")

    # Ensure all selected flights exist in either variables2_2 or variables2_1
    valid_variables = []
    missing_keys = []
    print(flight)
    for flight in selected_flight_2_members:
        if flight in variables2_2:
            valid_variables.append(variables2_2[flight].varValue)  # Use varValue as per your requirement
        elif flight in variables2_1:
            valid_variables.append(variables2_1[flight].varValue)
        else:
            missing_keys.append(flight)


Index Group 0: [0, 32, 38, 50, 55, 67, 77, 83, 89, 96, 103]
Selected Flight 2 Members: ['Mo/MLE/102/CMB_Tu/CMB/173/BLR', 'We/BLR/174/CMB_Tu/CMB/173/BLR', 'We/MLE/102/CMB_Tu/CMB/173/BLR', 'Th/BLR/174/CMB_Tu/CMB/173/BLR', 'Th/MLE/102/CMB_Tu/CMB/173/BLR', 'Fr/MLE/102/CMB_Tu/CMB/173/BLR', 'Sa/BLR/174/CMB_Tu/CMB/173/BLR', 'Sa/MLE/102/CMB_Tu/CMB/173/BLR', 'Sa/LHR/506/CMB_Tu/CMB/173/BLR', 'Su/MLE/102/CMB_Tu/CMB/173/BLR', 'Su/LHR/506/CMB_Tu/CMB/173/BLR']
Su/LHR/506/CMB_Sa/CMB/173/BLR
Index Group 1: [1, 33, 39, 51, 56, 68, 78, 84, 90, 97, 104]
Selected Flight 2 Members: ['Mo/MLE/102/CMB_Tu/CMB/306/SIN', 'We/BLR/174/CMB_Tu/CMB/306/SIN', 'We/MLE/102/CMB_Tu/CMB/306/SIN', 'Th/BLR/174/CMB_Tu/CMB/306/SIN', 'Th/MLE/102/CMB_Tu/CMB/306/SIN', 'Fr/MLE/102/CMB_Tu/CMB/306/SIN', 'Sa/BLR/174/CMB_Tu/CMB/306/SIN', 'Sa/MLE/102/CMB_Tu/CMB/306/SIN', 'Sa/LHR/506/CMB_Tu/CMB/306/SIN', 'Su/MLE/102/CMB_Tu/CMB/306/SIN', 'Su/LHR/506/CMB_Tu/CMB/306/SIN']
Su/LHR/506/CMB_Tu/CMB/173/BLR
Index Group 2: [2, 12, 23, 52, 57, 69,

In [ ]:
max_limit = merged_df['Flight_1_Max'].tolist()
min_limit = merged_df['Flight_1_Min'].tolist()

# Ensure `flight1_1` maps correctly to its respective LpVariable
flight1_in = []
for flight1_new in merged_df['extracted_part']:
    if flight1_new in user_inputs1:  # Assuming variables2_1 holds relevant LpVariables
        flight1_in.append(user_inputs1[flight1_new])
    else:
        flight1_in.append(0)  # Handle missing cases safely

# Loop over each row in df_unique to add constraints dynamically
for i, row in df_unique.iterrows():
    indices = row['full_part_index']
    if isinstance(indices, str):
        indices = list(map(int, indices.split(',')))

    # Extract selected flight members
    selected_flight_2_members = first_second_selected.loc[indices, "Flight_2_Allocated_Members"].dropna()

    valid_variables = []
    for flight in selected_flight_2_members:
        flight = flight.strip()
        if flight in variables2_2:
            valid_variables.append(variables2_2[flight])
        elif flight in variables2_1:
            valid_variables.append(variables2_1[flight])

    # Ensure valid index for flight1_in
    flight1_var = flight1_in[i] if i < len(flight1_in) else 0

    if valid_variables:
        upper_constraint_name = f"Constraint_{36 + i}_Upper"
        lower_constraint_name = f"Constraint_{36 + i}_Lower"

        # problem += sum(valid_variables) + flight1_var <= max_limit[i], upper_constraint_name
        # if min_limit[i] - flight1_var >= 0:
        #     problem += sum(valid_variables) + flight1_var >= min_limit[i], lower_constraint_name
        # else:
        #   problem += sum(valid_variables) >= 1, lower_constraint_name

        problem += sum(valid_variables)<= max_limit[i], upper_constraint_name

        problem += sum(valid_variables) >= min_limit[i], lower_constraint_name


In [ ]:
indices_2nd = []

for i in range(len(one_to_2_3_indices_dict)):
    row_indices = one_to_2_3_indices_dict["Indices"][i]
    indices_2nd.append(row_indices)

print(indices_2nd)

[[0, 32], [1, 33], [2, 34, 35, 36], [3, 37], [4, 38, 39, 40, 41], [5, 42, 43], [6, 44], [7, 45], [8, 46, 47], [9, 48], [10, 49, 50], [11, 51], [12, 52], [13, 53, 54, 55, 56, 57], [14, 58], [15, 59], [16, 60, 61], [17, 62, 63, 64, 65, 66], [18, 67, 68, 69], [19, 70, 71], [20, 72, 73, 74, 75, 76], [21, 77, 78, 79, 80, 81], [22, 82], [23, 83], [24, 84], [25, 85], [26, 86], [27, 87], [28, 88], [29, 89], [30, 90], [31, 91]]


In [ ]:
user_inputs1

{'Mo/MLE/102/CMB': 35,
 'Tu/CMB/173/BLR': 30,
 'Tu/CMB/306/SIN': 28,
 'Tu/BLR/174/CMB': 36,
 'Tu/SIN/307/MLE': 23,
 'Tu/MLE/102/CMB': 25,
 'We/CMB/173/BLR': 24,
 'We/BLR/174/CMB': 26,
 'We/MLE/102/CMB': 27,
 'Th/CMB/173/BLR': 23,
 'Th/CMB/306/SIN': 38,
 'Th/BLR/174/CMB': 23,
 'Th/MLE/102/CMB': 39,
 'Fr/CMB/306/SIN': 24,
 'Fr/SIN/307/MLE': 36,
 'Fr/MLE/102/CMB': 34,
 'Sa/CMB/173/BLR': 23,
 'Sa/BLR/174/CMB': 24,
 'Sa/MLE/102/CMB': 36,
 'Sa/LHR/506/CMB': 32,
 'Su/MLE/102/CMB': 38,
 'Su/LHR/506/CMB': 39}

In [ ]:
total_sum = sum(user_inputs1.values())
print("Total Sum:", total_sum)

Total Sum: 663


In [ ]:
# Define constraint to enforce total assigned value == user input
problem += (
    lpSum(variables2_2.values()) + lpSum(variables2_1.values()) == total_sum
), "Total Assignment Equality"

# Solve the problem
problem.solve()

# # Check the results
# total_assigned_value == sum(var.varValue for var in variables2_2.values()) + sum(var.varValue for var in variables2_1.values())

# print("Total Sum (Expected):", total_sum)  # Should be 149
# print("Total Assigned Value (Optimized):", total_assigned_value)  # Should also be 149


-1

In [ ]:
problem

Crew_Optimization:
MINIMIZE
2634.9*Fr_CMB_306_SIN_Tu_SIN_307_MLE + 709.9*Fr_MLE_102_CMB_Sa_CMB_173_BLR + 784.9*Fr_MLE_102_CMB_Th_CMB_173_BLR + 2679.9*Fr_MLE_102_CMB_Th_CMB_306_SIN + 754.9*Fr_MLE_102_CMB_Tu_CMB_173_BLR + 2649.9*Fr_MLE_102_CMB_Tu_CMB_306_SIN + 769.9*Fr_MLE_102_CMB_We_CMB_173_BLR + 2624.9*Fr_SIN_307_MLE_Mo_MLE_102_CMB + 2619.9*Fr_SIN_307_MLE_Sa_MLE_102_CMB + 2619.9*Fr_SIN_307_MLE_Su_MLE_102_CMB + 1597.4*Fr_SIN_307_MLE_Th_MLE_102_CMB + 2639.9*Fr_SIN_307_MLE_Tu_MLE_102_CMB + 3447.4*Fr_SIN_307_MLE_We_MLE_102_CMB + 2649.9*Mo_MLE_102_CMB_Fr_CMB_306_SIN + 769.9*Mo_MLE_102_CMB_Sa_CMB_173_BLR + 739.9*Mo_MLE_102_CMB_Th_CMB_173_BLR + 2604.9*Mo_MLE_102_CMB_Th_CMB_306_SIN + 709.9*Mo_MLE_102_CMB_Tu_CMB_173_BLR + 2619.9*Mo_MLE_102_CMB_Tu_CMB_306_SIN + 724.9*Mo_MLE_102_CMB_We_CMB_173_BLR + 2699.9*Sa_BLR_174_CMB_Fr_CMB_306_SIN + 789.9*Sa_BLR_174_CMB_Th_CMB_173_BLR + 3462.4*Sa_BLR_174_CMB_Th_CMB_306_SIN + 759.9*Sa_BLR_174_CMB_Tu_CMB_173_BLR + 1537.4*Sa_BLR_174_CMB_Tu_CMB_306_SIN + 774.9*S

In [ ]:
from pulp import PULP_CBC_CMD
problem.solve(PULP_CBC_CMD(msg=True))  # Use solver output to debug


-1

In [ ]:
problem.solve()

-1

In [ ]:
from pulp import LpStatus

print("Solver Status:", LpStatus[problem.status])

Solver Status: Infeasible


In [ ]:
# # Check if the LP problem is feasible
# if problem.status != 1:  # status 1 means optimal, other values indicate infeasibility or other issues
#     print("The LP problem is infeasible.")
#     # To get the minimum value of the relevant variables, you can loop through them
#     for var in all_variables:
#         if var in variables2_2:
#             # If the variable is in variables2_2 and the problem is infeasible, get its minimum value
#             print(f"{var}: {variables2_2[var].lowBound}")  # Using lowBound to get the minimum value
#         elif var in variables2_1:
#             print(f"{var}: {variables2_1[var].lowBound}")
# else:
print("Optimal Solution:")
all_variables = set(variables2_1) | set(variables2_2)  # Get all unique variables from both lists

for var in all_variables:
    if var in variables2_2:
        print(f"{var}: {variables2_2[var].varValue}")
    elif var in variables2_1:
        print(f"{var}: {variables2_1[var].varValue}")


Optimal Solution:
Sa/CMB/173/BLR_We/BLR/174/CMB: 0.0
Mo/MLE/102/CMB_Sa/CMB/173/BLR: 0.0
Mo/MLE/102/CMB_Th/CMB/173/BLR: 0.0
We/MLE/102/CMB_Sa/CMB/173/BLR: 0.0
Su/LHR/506/CMB_Fr/CMB/306/SIN: 0.0
Fr/SIN/307/MLE_Su/MLE/102/CMB: 6.0
Tu/MLE/102/CMB_Sa/CMB/173/BLR: 0.0
Sa/LHR/506/CMB_Sa/CMB/173/BLR: 0.0
Sa/CMB/173/BLR_Th/BLR/174/CMB: 0.0
Tu/BLR/174/CMB_Sa/CMB/173/BLR: 0.0
Su/MLE/102/CMB_Tu/CMB/306/SIN: 2.0
We/BLR/174/CMB_Th/CMB/173/BLR: 25.0
Sa/MLE/102/CMB_Tu/CMB/173/BLR: 19.0
Fr/SIN/307/MLE_Sa/MLE/102/CMB: 6.0
We/CMB/173/BLR_Sa/BLR/174/CMB: 0.0
Su/LHR/506/CMB_Th/CMB/306/SIN: 2.0
Th/CMB/306/SIN_Tu/SIN/307/MLE: 24.0
Sa/LHR/506/CMB_Tu/CMB/173/BLR: 0.0
Tu/MLE/102/CMB_Fr/CMB/306/SIN: 0.0
Sa/LHR/506/CMB_Th/CMB/173/BLR: 0.0
We/MLE/102/CMB_Th/CMB/306/SIN: 3.0
Fr/MLE/102/CMB_Tu/CMB/173/BLR: 0.0
Mo/MLE/102/CMB_Th/CMB/306/SIN: 2.0
Sa/BLR/174/CMB_Th/CMB/306/SIN: 2.0
Mo/MLE/102/CMB_Tu/CMB/173/BLR: 5.0
We/MLE/102/CMB_Tu/CMB/306/SIN: 0.0
Tu/CMB/173/BLR_We/BLR/174/CMB: 0.0
Tu/BLR/174/CMB_Fr/CMB/306/SIN: 0.0

In [ ]:
total_assigned_value = 0

for var in all_variables:
    if var in variables2_2:
        total_assigned_value += variables2_2[var].varValue
    elif var in variables2_1:
        total_assigned_value += variables2_1[var].varValue

print("Total Assigned Value:", total_assigned_value)

Total Assigned Value: 663.0


In [ ]:
# Ensure that the problem has been solved before accessing the objective value
if hasattr(problem, 'objective') and problem.objective is not None:
    print(f"Total Cost: {problem.objective.value()}")
else:
    print("Objective value is not available.")

Total Cost: 1120726.2000000004


In [ ]:
def get_optimal_values(variables1, variables2):
    # Merge both variable dictionaries
    all_variables = {**variables1, **variables2}  # Merge variables2_1 and variables2_2
    return {var: all_variables[var].varValue for var in all_variables}

optimal_values = get_optimal_values(variables2_1, variables2_2)
print("Optimal Values:", optimal_values)
print("Total Cost:", problem.objective.value())


Optimal Values: {'Mo/MLE/102/CMB_Tu/CMB/173/BLR': 5.0, 'Mo/MLE/102/CMB_We/CMB/173/BLR': 0.0, 'Mo/MLE/102/CMB_Th/CMB/173/BLR': 0.0, 'Mo/MLE/102/CMB_Fr/CMB/306/SIN': 26.0, 'Mo/MLE/102/CMB_Sa/CMB/173/BLR': 0.0, 'Tu/CMB/173/BLR_Tu/BLR/174/CMB': 10.0, 'Tu/CMB/173/BLR_We/BLR/174/CMB': 0.0, 'Tu/CMB/173/BLR_Th/BLR/174/CMB': 0.0, 'Tu/CMB/173/BLR_Sa/BLR/174/CMB': 0.0, 'Tu/BLR/174/CMB_We/CMB/173/BLR': 33.0, 'Tu/BLR/174/CMB_Th/CMB/173/BLR': 0.0, 'Tu/BLR/174/CMB_Fr/CMB/306/SIN': 0.0, 'Tu/BLR/174/CMB_Sa/CMB/173/BLR': 0.0, 'Tu/SIN/307/MLE_Mo/MLE/102/CMB': 0.0, 'Tu/SIN/307/MLE_Sa/MLE/102/CMB': 0.0, 'Tu/SIN/307/MLE_Su/MLE/102/CMB': 0.0, 'Tu/MLE/102/CMB_We/CMB/173/BLR': 5.0, 'Tu/MLE/102/CMB_Th/CMB/173/BLR': 0.0, 'Tu/MLE/102/CMB_Fr/CMB/306/SIN': 0.0, 'Tu/MLE/102/CMB_Sa/CMB/173/BLR': 0.0, 'We/CMB/173/BLR_Tu/BLR/174/CMB': 0.0, 'We/CMB/173/BLR_We/BLR/174/CMB': 35.0, 'We/CMB/173/BLR_Th/BLR/174/CMB': 0.0, 'We/CMB/173/BLR_Sa/BLR/174/CMB': 0.0, 'We/BLR/174/CMB_Tu/CMB/173/BLR': 0.0, 'We/BLR/174/CMB_Tu/CMB/306/SI

In [ ]:
flight_3_members

['Mo/MLE/102/CMB_Tu/CMB/306/SIN_NaN',
 'Mo/MLE/102/CMB_Th/CMB/306/SIN_NaN',
 'Tu/CMB/306/SIN_Fr/SIN/307/MLE_NaN',
 'Tu/BLR/174/CMB_Th/CMB/306/SIN_NaN',
 'Tu/SIN/307/MLE_We/MLE/102/CMB_NaN',
 'Tu/SIN/307/MLE_Th/MLE/102/CMB_NaN',
 'Tu/SIN/307/MLE_Fr/MLE/102/CMB_NaN',
 'Tu/MLE/102/CMB_Th/CMB/306/SIN_NaN',
 'We/BLR/174/CMB_Th/CMB/306/SIN_NaN',
 'We/BLR/174/CMB_Fr/CMB/306/SIN_NaN',
 'We/MLE/102/CMB_Th/CMB/306/SIN_NaN',
 'We/MLE/102/CMB_Fr/CMB/306/SIN_NaN',
 'Th/CMB/306/SIN_Tu/SIN/307/MLE_NaN',
 'Th/CMB/306/SIN_Fr/SIN/307/MLE_NaN',
 'Th/BLR/174/CMB_Fr/CMB/306/SIN_NaN',
 'Th/MLE/102/CMB_Fr/CMB/306/SIN_NaN',
 'Fr/CMB/306/SIN_Tu/SIN/307/MLE_NaN',
 'Fr/SIN/307/MLE_Mo/MLE/102/CMB_NaN',
 'Fr/SIN/307/MLE_Tu/MLE/102/CMB_NaN',
 'Fr/SIN/307/MLE_We/MLE/102/CMB_NaN',
 'Fr/SIN/307/MLE_Sa/MLE/102/CMB_NaN',
 'Fr/SIN/307/MLE_Su/MLE/102/CMB_NaN',
 'Sa/BLR/174/CMB_Tu/CMB/306/SIN_NaN',
 'Sa/BLR/174/CMB_Th/CMB/306/SIN_NaN',
 'Sa/MLE/102/CMB_Tu/CMB/306/SIN_NaN',
 'Sa/MLE/102/CMB_Th/CMB/306/SIN_NaN',
 'Sa/LHR/506

In [ ]:
variables3 = {
    flight_3_members[var]: LpVariable(
        flight_3_members[var],
        lowBound=0,
        cat='Integer'
    )
    for var in range(len(flight_3_members))
}

In [ ]:
variables3

{'Mo/MLE/102/CMB_Tu/CMB/306/SIN_NaN': Mo_MLE_102_CMB_Tu_CMB_306_SIN_NaN,
 'Mo/MLE/102/CMB_Th/CMB/306/SIN_NaN': Mo_MLE_102_CMB_Th_CMB_306_SIN_NaN,
 'Tu/CMB/306/SIN_Fr/SIN/307/MLE_NaN': Tu_CMB_306_SIN_Fr_SIN_307_MLE_NaN,
 'Tu/BLR/174/CMB_Th/CMB/306/SIN_NaN': Tu_BLR_174_CMB_Th_CMB_306_SIN_NaN,
 'Tu/SIN/307/MLE_We/MLE/102/CMB_NaN': Tu_SIN_307_MLE_We_MLE_102_CMB_NaN,
 'Tu/SIN/307/MLE_Th/MLE/102/CMB_NaN': Tu_SIN_307_MLE_Th_MLE_102_CMB_NaN,
 'Tu/SIN/307/MLE_Fr/MLE/102/CMB_NaN': Tu_SIN_307_MLE_Fr_MLE_102_CMB_NaN,
 'Tu/MLE/102/CMB_Th/CMB/306/SIN_NaN': Tu_MLE_102_CMB_Th_CMB_306_SIN_NaN,
 'We/BLR/174/CMB_Th/CMB/306/SIN_NaN': We_BLR_174_CMB_Th_CMB_306_SIN_NaN,
 'We/BLR/174/CMB_Fr/CMB/306/SIN_NaN': We_BLR_174_CMB_Fr_CMB_306_SIN_NaN,
 'We/MLE/102/CMB_Th/CMB/306/SIN_NaN': We_MLE_102_CMB_Th_CMB_306_SIN_NaN,
 'We/MLE/102/CMB_Fr/CMB/306/SIN_NaN': We_MLE_102_CMB_Fr_CMB_306_SIN_NaN,
 'Th/CMB/306/SIN_Tu/SIN/307/MLE_NaN': Th_CMB_306_SIN_Tu_SIN_307_MLE_NaN,
 'Th/CMB/306/SIN_Fr/SIN/307/MLE_NaN': Th_CMB_306_SI

In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
# Define the problem
problem1 = LpProblem("Crew_Optimization2", LpMinimize)

In [ ]:
# Define weights for objective function
alpha = 1     # Weight for cost minimization
beta = -0.1   # Weight to encourage maximizing crew utilization

problem1 += alpha*lpSum([salary_flight_3[i] * variables3[flight_3_members[i]] for i in set(range(len(flight_3_members)))]) + beta*lpSum(variables3.values())

In [ ]:
for i, indices in enumerate(indices_2nd):
    selected_flight_3_members = first_second_third_selected.loc[indices, 'Flight_3_Allocated_Members']

    # Debugging print
    print(f"Index Group {i}: {indices}")
    print(f"Selected Flight 3 Members: {selected_flight_3_members.tolist()}")

    # Ensure all selected flights exist in either variables2 or variables4
    valid_variables = []
    missing_keys = []

    for flight in selected_flight_3_members:
        if flight in variables3:
            valid_variables.append(variables3[flight])
        else:
            missing_keys.append(flight)

    # Ensure Flight_1_Allocated_Members exist in optimal_values
    flight_2_key = first_second_third_selected['Flight_2_Allocated_Members'].iloc[i]
    if flight_2_key not in optimal_values:
        print(f"Warning: Key {flight_1_key} missing from optimal_values")
        continue  # Skip this constraint if optimal_values key is missing

    # Print missing variables for debugging
    if missing_keys:
        print(f"Warning: These flights are missing from both variables2 and variables4: {missing_keys}")

    # Only create constraint if valid variables exist
    if valid_variables:
        # problem1 += lpSum(valid_variables) == optimal_values[flight_2_key]
          problem1 += lpSum(valid_variables) >=1

Index Group 0: [0, 32]
Selected Flight 3 Members: ['Mo/MLE/102/CMB_Tu/CMB/306/SIN_NaN', 'Mo/MLE/102/CMB_Tu/CMB/306/SIN_Fr/SIN/307/MLE']
Index Group 1: [1, 33]
Selected Flight 3 Members: ['Mo/MLE/102/CMB_Th/CMB/306/SIN_NaN', 'Mo/MLE/102/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE']
Index Group 2: [2, 34, 35, 36]
Selected Flight 3 Members: ['Tu/CMB/306/SIN_Fr/SIN/307/MLE_NaN', 'Tu/CMB/306/SIN_Fr/SIN/307/MLE_Mo/MLE/102/CMB', 'Tu/CMB/306/SIN_Fr/SIN/307/MLE_Sa/MLE/102/CMB', 'Tu/CMB/306/SIN_Fr/SIN/307/MLE_Su/MLE/102/CMB']
Index Group 3: [3, 37]
Selected Flight 3 Members: ['Tu/BLR/174/CMB_Th/CMB/306/SIN_NaN', 'Tu/BLR/174/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE']
Index Group 4: [4, 38, 39, 40, 41]
Selected Flight 3 Members: ['Tu/SIN/307/MLE_We/MLE/102/CMB_NaN', 'Tu/SIN/307/MLE_We/MLE/102/CMB_Th/CMB/173/BLR', 'Tu/SIN/307/MLE_We/MLE/102/CMB_Th/CMB/306/SIN', 'Tu/SIN/307/MLE_We/MLE/102/CMB_Fr/CMB/306/SIN', 'Tu/SIN/307/MLE_We/MLE/102/CMB_Sa/CMB/173/BLR']
Index Group 5: [5, 42, 43]
Selected Flight 3 Members: ['Tu/S

In [ ]:
problem1

Crew_Optimization2:
MINIMIZE
3487.4*Fr_CMB_306_SIN_Tu_SIN_307_MLE_NaN + 3832.4*Fr_CMB_306_SIN_Tu_SIN_307_MLE_Th_MLE_102_CMB + 3817.4*Fr_CMB_306_SIN_Tu_SIN_307_MLE_We_MLE_102_CMB + 1577.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_NaN + 1962.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_Th_CMB_173_BLR + 3832.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_Th_CMB_306_SIN + 1932.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_Tu_CMB_173_BLR + 3802.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_Tu_CMB_306_SIN + 1947.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_We_CMB_173_BLR + 1547.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_NaN + 1962.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_Th_CMB_173_BLR + 3832.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_Th_CMB_306_SIN + 1932.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_Tu_CMB_173_BLR + 3802.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_Tu_CMB_306_SIN + 1947.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_We_CMB_173_BLR + 1562.4*Fr_SIN_307_MLE_Su_MLE_102_CMB_NaN + 1962.4*Fr_SIN_307_MLE_Su_MLE_102_CMB_Th_CMB_173_BLR + 3832.4*Fr_SIN_307_MLE_Su_MLE_102_CMB_Th_CMB_306_SIN + 1932.4*Fr_SIN_307_MLE_Su_MLE_102_CMB_Tu_CMB

In [ ]:
# Filter out rows where 'NaN' is part of the 'Flight_3_Allocated_Members' value
df = pd.DataFrame(flight_3_members, columns=["Flight_3_Allocated_Members"])
df = df[~df["Flight_3_Allocated_Members"].str.contains("NaN", na=False)]

# Function to extract substring after "_"
def extract_after_underscore(value):
    if isinstance(value, str) and "_" in value:
        return value.split("_", 2)[1]  # Get part after "_"
    return None  # Return None if "_" is not found

# Extract the second part
df['extracted_part'] = df["Flight_3_Allocated_Members"].apply(extract_after_underscore)

# Drop duplicate extracted parts but keep first occurrence
df_unique = df[['extracted_part']].drop_duplicates().reset_index(drop=True)

# Get the indices where each extracted part appears (ensuring exact suffix matching)
df_unique['full_part_index'] = df_unique['extracted_part'].apply(
    lambda x: df.index[df['Flight_3_Allocated_Members'].apply(
        lambda y: y.endswith(x) if isinstance(y, str) else False
    )].tolist()
)

# Get the count of occurrences for each extracted part
df_unique['count'] = df_unique['full_part_index'].apply(len)

# Display the result
df_unique


,extracted_part,full_part_index,count
0,Tu/CMB/306/SIN,"[63, 73, 78]",3
1,Th/CMB/306/SIN,"[39, 66, 69, 71, 76, 81]",6
2,Fr/SIN/307/MLE,"[32, 33, 37, 45, 47, 50, 82, 83, 84, 85, 86, 8...",16
3,We/MLE/102/CMB,"[52, 55, 60]",3
4,Th/MLE/102/CMB,[61],1
5,Fr/MLE/102/CMB,[],0
6,Fr/CMB/306/SIN,"[40, 42]",2
7,Tu/SIN/307/MLE,"[46, 48, 49, 51, 58, 59]",6
8,Mo/MLE/102/CMB,"[34, 53]",2
9,Tu/MLE/102/CMB,[54],1


In [ ]:
# Merge on extracted_part (df_unique) and Members (flight1_details)
merged_df = df_unique.merge(flight1_details, left_on='extracted_part', right_on='Flight_1_Allocated_Members', how='left')

# Drop the 'Members' column and any other columns from flight1_details you don't want
merged_df = merged_df.drop(columns=['Flight_1_Allocated_Members', 'Total_salary'])

# Display the result
merged_df

,extracted_part,full_part_index,count,Flight_1_Min,Flight_1_Max
0,Tu/CMB/306/SIN,"[63, 73, 78]",3,3,50
1,Th/CMB/306/SIN,"[39, 66, 69, 71, 76, 81]",6,3,50
2,Fr/SIN/307/MLE,"[32, 33, 37, 45, 47, 50, 82, 83, 84, 85, 86, 8...",16,6,50
3,We/MLE/102/CMB,"[52, 55, 60]",3,6,50
4,Th/MLE/102/CMB,[61],1,6,50
5,Fr/MLE/102/CMB,[],0,6,50
6,Fr/CMB/306/SIN,"[40, 42]",2,3,50
7,Tu/SIN/307/MLE,"[46, 48, 49, 51, 58, 59]",6,6,50
8,Mo/MLE/102/CMB,"[34, 53]",2,6,50
9,Tu/MLE/102/CMB,[54],1,6,50


In [ ]:
from pulp import lpSum

for i, row in df_unique.iterrows():
    extracted_part = row['extracted_part']
    print(type(indices))
    if not indices:
        continue  # Skip if no matching indices

    # Extract selected flight members based on the indices
    selected_flight_3_members = first_second_third_selected.loc[indices, "Flight_3_Allocated_Members"]
    # Debugging print
    print(f"Index Group {i}: {indices}")
    print(f"Selected Flight 2 Members: {selected_flight_3_members.tolist()}")

    # Ensure all selected flights exist in either variables2_2 or variables2_1
    valid_variables = []
    missing_keys = []
    print(flight)
    for flight in selected_flight_3_members:
        if flight in variables3:
            valid_variables.append(variables3[flight].varValue)  # Use varValue as per your requirement
        else:
            missing_keys.append(flight)


<class 'list'>
Index Group 0: [31, 91]
Selected Flight 2 Members: ['Su/LHR/506/CMB_Th/CMB/306/SIN_NaN', 'Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE']
Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE
<class 'list'>
Index Group 1: [31, 91]
Selected Flight 2 Members: ['Su/LHR/506/CMB_Th/CMB/306/SIN_NaN', 'Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE']
Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE
<class 'list'>
Index Group 2: [31, 91]
Selected Flight 2 Members: ['Su/LHR/506/CMB_Th/CMB/306/SIN_NaN', 'Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE']
Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE
<class 'list'>
Index Group 3: [31, 91]
Selected Flight 2 Members: ['Su/LHR/506/CMB_Th/CMB/306/SIN_NaN', 'Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE']
Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE
<class 'list'>
Index Group 4: [31, 91]
Selected Flight 2 Members: ['Su/LHR/506/CMB_Th/CMB/306/SIN_NaN', 'Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE']
Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE
<class 'list'>


In [ ]:
# Assuming merged_df, df_unique, first_second_selected, and variables3 are already defined

max_limit = []
# Populate max_limit with the count values from merged_df
for i in merged_df['Flight_1_Max']:
    max_limit.append(i)

min_limit = []
# Populate min_limit with the count values from merged_df
for i in merged_df['Flight_1_Min']:
    min_limit.append(i)

# Loop over each row in df_unique to add constraints dynamically
for i, row in df_unique.iterrows():
    # Convert full_part_index from string to list of integers
    indices = row['full_part_index']
    if isinstance(indices, str):
        indices = list(map(int, indices.split(',')))

    # Extract selected flight members based on the indices
    selected_flight_3_members = first_second_third_selected.loc[indices, "Flight_3_Allocated_Members"].dropna()

    # Create a list to store valid variables
    valid_variables = []
    for flight in selected_flight_3_members:
        flight = flight.strip()  # Remove leading/trailing spaces
        # Check if the flight is in variables3 and add the corresponding LpVariable
        if flight in variables3:
            valid_variables.append(variables3[flight])  # Add the LpVariable from variables3

    # If there are valid variables, add constraints
    if valid_variables:
        # Ensure unique constraint names by using `i` to generate the name
        upper_constraint_name = f"Constraint_{i}_Upper_{hash(tuple(valid_variables))}"
        lower_constraint_name = f"Constraint_{i}_Lower_{hash(tuple(valid_variables))}"

        # Add the upper bound constraint
        problem1 += sum(valid_variables)  <= max_limit[i], upper_constraint_name

        # Add the lower bound constraint
        problem1 += sum(valid_variables) >= min_limit[i], lower_constraint_name


In [ ]:
problem1

Crew_Optimization2:
MINIMIZE
3487.4*Fr_CMB_306_SIN_Tu_SIN_307_MLE_NaN + 3832.4*Fr_CMB_306_SIN_Tu_SIN_307_MLE_Th_MLE_102_CMB + 3817.4*Fr_CMB_306_SIN_Tu_SIN_307_MLE_We_MLE_102_CMB + 1577.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_NaN + 1962.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_Th_CMB_173_BLR + 3832.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_Th_CMB_306_SIN + 1932.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_Tu_CMB_173_BLR + 3802.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_Tu_CMB_306_SIN + 1947.4*Fr_SIN_307_MLE_Mo_MLE_102_CMB_We_CMB_173_BLR + 1547.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_NaN + 1962.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_Th_CMB_173_BLR + 3832.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_Th_CMB_306_SIN + 1932.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_Tu_CMB_173_BLR + 3802.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_Tu_CMB_306_SIN + 1947.4*Fr_SIN_307_MLE_Sa_MLE_102_CMB_We_CMB_173_BLR + 1562.4*Fr_SIN_307_MLE_Su_MLE_102_CMB_NaN + 1962.4*Fr_SIN_307_MLE_Su_MLE_102_CMB_Th_CMB_173_BLR + 3832.4*Fr_SIN_307_MLE_Su_MLE_102_CMB_Th_CMB_306_SIN + 1932.4*Fr_SIN_307_MLE_Su_MLE_102_CMB_Tu_CMB

In [ ]:
problem1.solve()
from pulp import LpStatus

print("Solver Status:", LpStatus[problem1.status])

Solver Status: Optimal


In [ ]:
336
35
2282927
2221
26222# Print the results
print("Optimal Solution:")
for var in variables3:
    print(f"{var}: {variables3[var].varValue}")
print("Total Cost:", problem1.objective.value())

Optimal Solution:
Mo/MLE/102/CMB_Tu/CMB/306/SIN_NaN: 1.0
Mo/MLE/102/CMB_Th/CMB/306/SIN_NaN: 0.0
Tu/CMB/306/SIN_Fr/SIN/307/MLE_NaN: 0.0
Tu/BLR/174/CMB_Th/CMB/306/SIN_NaN: 1.0
Tu/SIN/307/MLE_We/MLE/102/CMB_NaN: 0.0
Tu/SIN/307/MLE_Th/MLE/102/CMB_NaN: 0.0
Tu/SIN/307/MLE_Fr/MLE/102/CMB_NaN: 1.0
Tu/MLE/102/CMB_Th/CMB/306/SIN_NaN: 0.0
We/BLR/174/CMB_Th/CMB/306/SIN_NaN: 0.0
We/BLR/174/CMB_Fr/CMB/306/SIN_NaN: 0.0
We/MLE/102/CMB_Th/CMB/306/SIN_NaN: 0.0
We/MLE/102/CMB_Fr/CMB/306/SIN_NaN: 0.0
Th/CMB/306/SIN_Tu/SIN/307/MLE_NaN: 0.0
Th/CMB/306/SIN_Fr/SIN/307/MLE_NaN: 0.0
Th/BLR/174/CMB_Fr/CMB/306/SIN_NaN: 0.0
Th/MLE/102/CMB_Fr/CMB/306/SIN_NaN: 0.0
Fr/CMB/306/SIN_Tu/SIN/307/MLE_NaN: 0.0
Fr/SIN/307/MLE_Mo/MLE/102/CMB_NaN: 0.0
Fr/SIN/307/MLE_Tu/MLE/102/CMB_NaN: 0.0
Fr/SIN/307/MLE_We/MLE/102/CMB_NaN: 0.0
Fr/SIN/307/MLE_Sa/MLE/102/CMB_NaN: 0.0
Fr/SIN/307/MLE_Su/MLE/102/CMB_NaN: 0.0
Sa/BLR/174/CMB_Tu/CMB/306/SIN_NaN: 1.0
Sa/BLR/174/CMB_Th/CMB/306/SIN_NaN: 1.0
Sa/MLE/102/CMB_Tu/CMB/306/SIN_NaN: 1.0
Sa/MLE/

In [ ]:
# Combine all variables for easier handling
all_variables = set(variables2_1) | set(variables2_2)

# Check if the LP problem is feasible
# if problem.status != 1:  # status 1 means optimal
#     print("The LP problem is infeasible.")
#     # Print the lower bound of relevant variables
#     for var in all_variables:
#         if var in variables2_2:
#             print(f"{var}: {variables2_2[var].lowBound}")
#         elif var in variables2_1:
#             print(f"{var}: {variables2_1[var].lowBound}")
# else:
# Create a list to store the variable names and their optimal values
solution_data = []

# Store optimal values for all variables
for var in all_variables:
    if var in variables2_2:
        solution_data.append({'Variable': var, 'Optimal_Value': variables2_2[var].varValue})
    elif var in variables2_1:
        solution_data.append({'Variable': var, 'Optimal_Value': variables2_1[var].varValue})

# Also store values for variables3
for var in variables3:
    solution_data.append({'Variable': var, 'Optimal_Value': variables3[var].varValue})

# Create a DataFrame
solution_df = pd.DataFrame(solution_data)

# Display the DataFrame
print(solution_df)

print("Total_Cost:", problem1.objective.value() + problem.objective.value())


                                         Variable  Optimal_Value
0                   Sa/CMB/173/BLR_We/BLR/174/CMB            0.0
1                   Mo/MLE/102/CMB_Sa/CMB/173/BLR            0.0
2                   Mo/MLE/102/CMB_Th/CMB/173/BLR            0.0
3                   We/MLE/102/CMB_Sa/CMB/173/BLR            0.0
4                   Su/LHR/506/CMB_Fr/CMB/306/SIN            0.0
..                                            ...            ...
197  Sa/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE            1.0
198  Su/MLE/102/CMB_Tu/CMB/306/SIN_Fr/SIN/307/MLE            0.0
199  Su/MLE/102/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE            1.0
200  Su/LHR/506/CMB_Tu/CMB/306/SIN_Fr/SIN/307/MLE            0.0
201  Su/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE            1.0

[202 rows x 2 columns]
Total_Cost: 1369714.6000000003


In [ ]:
solution_df

,Variable,Optimal_Value
0,Sa/CMB/173/BLR_We/BLR/174/CMB,0.0
1,Mo/MLE/102/CMB_Sa/CMB/173/BLR,0.0
2,Mo/MLE/102/CMB_Th/CMB/173/BLR,0.0
3,We/MLE/102/CMB_Sa/CMB/173/BLR,0.0
4,Su/LHR/506/CMB_Fr/CMB/306/SIN,0.0
...,...,...
197,Sa/LHR/506/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE,1.0
198,Su/MLE/102/CMB_Tu/CMB/306/SIN_Fr/SIN/307/MLE,0.0
199,Su/MLE/102/CMB_Th/CMB/306/SIN_Fr/SIN/307/MLE,1.0
200,Su/LHR/506/CMB_Tu/CMB/306/SIN_Fr/SIN/307/MLE,0.0


In [ ]:
final_df['Flight_1_Allocated_Members']

,Flight_1_Allocated_Members
0,Mo/MLE/102/CMB
1,Tu/CMB/173/BLR
2,Tu/CMB/306/SIN
3,Tu/BLR/174/CMB
4,Tu/SIN/307/MLE
5,Tu/MLE/102/CMB
6,We/CMB/173/BLR
7,We/BLR/174/CMB
8,We/MLE/102/CMB
9,Th/CMB/173/BLR


In [ ]:
type(selected_flight_2_members)

pandas.core.series.Series